Run this notebook to

- xx



## Import stuff and set parameteres

In [ ]:
%%javascript
(function(on) {
const e=$( "<a>Setup failed</a>" );
const ns="js_jupyter_suppress_warnings";
var cssrules=$("#"+ns);
if(!cssrules.length) cssrules = $("<style id='"+ns+"' type='text/css'>div.output_stderr { } </style>").appendTo("head");
e.click(function() {
    var s='Showing';  
    cssrules.empty()
    if(on) {
        s='Hiding';
        cssrules.append("div.output_stderr, div[data-mime-type*='.stderr'] { display:none; }");
    }
    e.text(s+' warnings (click to toggle)');
    on=!on;
}).click();
$(element).append(e);
})(true);

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import seaborn as sns
import pandas as pd
from scipy import stats
from scipy.stats import pearsonr 
from itertools import chain, zip_longest

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression
from sklearn.model_selection import KFold, cross_val_predict, train_test_split 
from sklearn.metrics import mean_squared_error, r2_score

import mne
from mne.stats import permutation_t_test
mne.set_log_level('warning') 

#%matplotlib qt
%matplotlib inline

input_dir = 'TaskresponseEpochsMastoids'


In [ ]:
def load_subj_eeg(path, file, downsample=None):
    fp = os.path.join(path, '%s-epo.fif' % file)
    print('>>> Loading %s' % fp)
    epochs = mne.read_epochs(fp, preload=True)
    if downsample is not None:
        epochs = epochs.resample(downsample)
    return epochs

def load_all_eeg(path, files, downsample=None):
    subject_epochs = [load_subj_eeg(path, file, downsample=downsample) for file in files]
    epochs = mne.epochs.concatenate_epochs(subject_epochs)
    return epochs

In [ ]:
participant_numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21] # 14 was excluded due to noise

sessions = [1, 2]

partners = ['"overconfident"', '"underconfident"']

epoch_start = 0.1
epoch_end = 0.6

## Linear regression predicting confidence

In [ ]:
%%capture

all_lr_scores = []

for subject in participant_numbers:   
#     print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % subject)
#     print('\033[0m')
    
    # merge response epochs from both sessions for the participant
    participant_files = []
    for session in sessions:
        participant_files.append('%i_%i' % (subject, session))
    epochs = load_all_eeg(path='%s/' % input_dir, files=participant_files)
    
    # crop epochs
    epochs = epochs.crop(tmin=epoch_start, tmax=epoch_end, include_tmax=True)
    
    # independent variables
    # get epoched data as a numpy array of shape (n_epochs, n_channels, n_times)
    X = epochs.get_data()
    
    n_epochs = X.shape[0]
    n_channels = X.shape[1]
    n_timepoints = X.shape[2]
    
    # collapse across trials and timepoints
    X = X.swapaxes(1, 2) # (n_epochs, n_times, n_channels)
    X = X.reshape(-1, n_channels) # (n_epochs x n_times, n_channels) --> n_samples, n_features

    # dependent variable as numpy array (n_epochs,)
    y = epochs.metadata['participant_confidence'].to_numpy()
    # extend from (n_epochs) to (n_epochs x n_timepoints, ) where confidence value stays the same
    y = np.repeat(y, n_timepoints)
    
    # fit regression
    reg = LinearRegression().fit(X, y)
    score = reg.score(X, y)
#     print(score)
    
#     # regression betas
#     betas = reg.coef_
#     #print(betas)
    
#     mne.viz.plot_topomap(data=betas, pos=epochs.info)

    all_lr_scores.append(score)


In [ ]:

print('Average R-squared linear regression: %f' % (sum(all_lr_scores) / len(all_lr_scores),))


## Cross-validation random split across timepoints

In [ ]:
all_cv_scores = []

for subject in participant_numbers:
    print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % subject)
    print('\033[0m')
    
    # merge response epochs from both sessions for the participant
    participant_files = []
    for session in sessions:
        participant_files.append('%i_%i' % (subject, session))
    epochs = load_all_eeg(path='%s/' % input_dir, files=participant_files)
    
    # crop epochs
    epochs = epochs.crop(tmin=epoch_start, tmax=epoch_end, include_tmax=True)
    
    
    
    ###### INDEPENDENT VARIABLES
    # get epoched data as a numpy array of shape (n_epochs, n_channels, n_times) (600, 64, 127) (250 Hz --> every 4 ms)
    X = epochs.get_data()
    n_channels = X.shape[1]
    n_timepoints = X.shape[2]
    X = X.swapaxes(1, 2) # (n_epochs, n_times, n_channels)

    
    
    ###### DEPENDENT VARIABLES
    ## this version is splitting data randomly by timepoints - topoplots etc look v similar
    # 50% training data, 50% test data
    X = X.reshape(-1, n_channels)
    y = epochs.metadata['participant_confidence'].to_numpy()
    y = np.repeat(y, n_timepoints)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    

    
    ###### TRAIN CLASSIFIER
    regressor = LinearRegression()  
    regressor.fit(X_train, y_train) #training the algorithm

    # regression betas
    betas = regressor.coef_
    
    # predict y from X_test
    y_pred = regressor.predict(X_test)    
    
    
    ###### PLOT
    # plot topoplot with beta weights
    mne.viz.plot_topomap(data=betas, pos=epochs.info)
    print("Beta weights for each channel \n\n")

    # plot topoplot with sensor projections
    # get out sensor projections (see Parra et al paper)
    y = y_pred
    x = X_test
    a =  y.T@x * ( 1 / (y.T@y) )
    mne.viz.plot_topomap(data=a, pos=epochs.info)
    print("Sensor projections \n\n")

    # plot confidence signal ERP over time (use betas rather than a weights)
    across_epochs_erp = epochs.average().get_data() # gives matrix with channels x timepoints (averaging across epochs)
    weighted_erp = across_epochs_erp * betas.reshape((betas.size, 1)) # weigh the voltages by the channel weights
    average_weighted_erp = weighted_erp.mean(axis=0) # compute the average across electrodes 
    x_axis = np.arange(epoch_start, epoch_end, (epoch_end-epoch_start)/(n_timepoints))
    plt.plot(x_axis, average_weighted_erp)
    plt.show();
    print("Average voltages (across epochs) weighted by regression betas for each electrode \n\n")
    
#     # plot normal ERP over time
#     across_epochs_erp = epochs.average().get_data() # gives matrix with channels x timepoints (averaging across epochs)
#     average_across_epochs_erp = across_epochs_erp.mean(axis=0) # compute the average across electrodes 
#     x_axis = np.arange(epoch_start, epoch_end, (epoch_end-epoch_start)/(n_timepoints))
#     plt.plot(x_axis, average_across_epochs_erp)
#     plt.show()
#     print("Average ERP (not weighted) \n\n")

    
    print('R-squared:', r2_score(y_test, y_pred))  
    all_cv_scores.append(r2_score(y_test, y_pred))
    
    

In [ ]:

print('Average R-squared cross validation random timepoint split: %f' % (sum(all_cv_scores) / len(all_cv_scores),))


## Cross-validation training on odd (even) trials, testing on even (odd) trials

In [ ]:
all_scores_test = []
all_scores_test_trialwise = []
all_scores_train = []
all_scores_train_trialwise = []


for subject in participant_numbers:
    print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % subject)
    print('\033[0m')
    
    
    # merge response epochs from both sessions for the participant
    participant_files = []
    for session in sessions:
        participant_files.append('%i_%i' % (subject, session))
    epochs = load_all_eeg(path='%s/' % input_dir, files=participant_files)
    
    # crop epochs
    epochs = epochs.crop(tmin=epoch_start, tmax=epoch_end, include_tmax=True)
    
    
    ######################################################################
    ####################### INDEPENDENT VARIABLE ######################### 
    ######################################################################
    
    # get epoched data as a numpy array of shape (n_epochs, n_channels, n_times) (600, 64, 127) (250 Hz --> every 4 ms)
    X = epochs.get_data()
    n_epochs = X.shape[0]
    n_channels = X.shape[1]
    n_timepoints = X.shape[2]
    X = X.swapaxes(1, 2) # (n_epochs, n_times, n_channels) (600, 127, 64)
    
    # divide into odd and even trials (3D array) (300, 127, 64)
    X_odd_trials = X[1::2, :, :] # select every second epoch (start:stop:step, :, :)
    X_even_trials = X[0::2, :, :] # select every other second epoch
    
    # collapse across trials and timepoints (2D array) (triple checked this and it is correctly sorted as e1t1, e1t2, e1t3, ...)
    X_odd_per_timepoint = X_odd_trials.reshape(-1, n_channels) # (n_epochs x n_times, n_channels) --> n_samples, n_features
    X_even_per_timepoint = X_even_trials.reshape(-1, n_channels) # (n_epochs x n_times, n_channels) --> n_samples, n_features

    
    ######################################################################
    ######################## DEPENDENT VARIABLE ########################## 
    ######################################################################
    
    # dependent variable as numpy array (n_epochs,)
    y = epochs.metadata['participant_confidence'].to_numpy()
    
    # divide into odd and even trials
    y_odd_trials = y[1::2]
    y_even_trials = y[0::2]
    
    # make from (n_epochs) to (n_epochs x n_timepoints, ) where confidence value stays the same
    y_odd_per_timepoint = np.repeat(y_odd_trials, n_timepoints)
    y_even_per_timepoint = np.repeat(y_even_trials, n_timepoints)
     
    
    ######################################################################
    ####################### TRAINING ON ODD TRIALS ####################### 
    ######################################################################
    
    print('\033[1m' + "\n\n ______ train on odd trials ______\n")
    print('\033[0m')
    
    X_train_trials = X_odd_trials
    X_train_per_timepoint = X_odd_per_timepoint
    
    X_test_trials = X_even_trials
    X_test_per_timepoint = X_even_per_timepoint
    
    
    y_train_trials = y_odd_trials
    y_train_per_timepoint = y_odd_per_timepoint
    
    y_test_trials = y_even_trials
    y_test_per_timepoint = y_even_per_timepoint
    
    
    
    # train classifier
    regressor = LinearRegression()  
    regressor.fit(X_train_per_timepoint, y_train_per_timepoint) #training the algorithm on every timepoint
    betas = regressor.coef_   
    intercept = regressor.intercept_
    
    # predict y from X_test 
    # OPTION 1
    # use predict function on individual timepoints
    y_pred_per_timepoint = regressor.predict(X_test_per_timepoint) 
    # then average predicted values within an epoch
    epoch_y_pred = np.mean(y_pred_per_timepoint.reshape(-1, n_timepoints), axis=1)
    
    # OPTION 2
    # use betas and intercept on individual timepoints   
    y_pred_per_timepoint_2 = np.sum(X_test_per_timepoint * betas, axis=1) + intercept 
    # then average predicted values within an epoch
    epoch_y_pred_2 = np.mean(y_pred_per_timepoint_2.reshape(-1, n_timepoints), axis=1)
    
    # OPTION 3
    # use predict function on epoch average
    epoch_y_pred_3 = regressor.predict(np.mean(X_test_trials, axis=1)) # take mean voltage along n_times axis (n_epochs, n_times, n_channels)
    
    # OPTION 4
    # use betas and intercept on epoch average
    epoch_y_pred_4 = np.sum(np.mean(X_test_trials, axis=1) * betas, axis=1) + intercept 

#     # sanity check - all options give the same predicted y per epoch
#     print(epoch_y_pred_1[0:6])
#     print(epoch_y_pred_2[0:6])
#     print(epoch_y_pred_3[0:6])
#     print(epoch_y_pred_4[0:6])


    print('\033[1m' + "\n test data")
    print('\033[0m')
    # R squared for timepoints
    print('R-squared timepoint data:', r2_score(y_test_per_timepoint, y_pred_per_timepoint))
#     # sanity check - manual computation gives same R-squared
#     actual_minus_predicted = sum((y_test_per_timepoint - y_pred_per_timepoint_1)**2)
#     actual_minus_actual_mean = sum((y_test_per_timepoint - y_test_per_timepoint.mean())**2)
#     r2 = 1 - actual_minus_predicted/actual_minus_actual_mean
#     print('R-squared timepoint data:', r2)
    
    # R squared for trials
    print('R-squared trial data:', r2_score(y_test_trials, epoch_y_pred))
#     # sanity check - manual computation gives same R-squared
#     actual_minus_predicted = sum((y_test_trials - epoch_y_pred_1)**2)
#     actual_minus_actual_mean = sum((y_test_trials - y_test_trials.mean())**2)
#     r2 = 1 - actual_minus_predicted/actual_minus_actual_mean
#     print('R-squared trial data:', r2)
    # plot y against predicted y
    sns.regplot(x=y_test_trials, y=epoch_y_pred, scatter_kws={'alpha':0.5})
    plt.xlabel('Reported confidence')
    plt.ylabel('Classifier prediction')
    plt.title('Reported trial confidence against predicted trial confidence test data')
    plt.show();
    

    # R squared for training data
    print('\033[1m' + "\n train data")
    print('\033[0m')
    # R squared for timepoints
    y_pred_per_timepoint_training = regressor.predict(X_train_per_timepoint) 
    print('R-squared training timepoint data:', r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))
    # R squared for trials
    epoch_y_pred_train = np.mean(y_pred_per_timepoint_training.reshape(-1, n_timepoints), axis=1)
    print('R-squared training trial data:', r2_score(y_train_trials, epoch_y_pred_train))
    # plot y against predicted y
    sns.regplot(x=y_train_trials, y=epoch_y_pred_train, scatter_kws={'alpha':0.5})
    plt.xlabel('Reported confidence')
    plt.ylabel('Classifier prediction')
    plt.title('Reported trial confidence against predicted trial confidence train data')
    plt.show();
    
    all_scores_test.append(r2_score(y_test_per_timepoint, y_pred_per_timepoint))
    all_scores_test_trialwise.append(r2_score(y_test_trials, epoch_y_pred))
    all_scores_train.append(r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))
    all_scores_train_trialwise.append(r2_score(y_train_trials, epoch_y_pred_train))
    

    ####################### PLOTTING ####################### 
    
    # plot topoplot with beta weights
    mne.viz.plot_topomap(data=betas, pos=epochs.info)
    print("Beta weights for each channel \n\n")

    
    # plot topoplot with sensor projections (see Parra et al paper for equation)
    y = y_pred_per_timepoint
    x = X_test_per_timepoint
    a =  y.T@x * ( 1 / (y.T@y) )
    mne.viz.plot_topomap(data=a, pos=epochs.info)
    print("Sensor projections \n\n")
    
#     # sanity check - plotting with trial data rather than timepoint data gives the same plot
#     y = epoch_y_pred
#     x = np.mean(X_test_trials, axis=1) 
#     a =  y.T@x * ( 1 / (y.T@y) )
#     mne.viz.plot_topomap(data=a, pos=epochs.info)
#     print("Sensor projections trial data \n\n")
    
    # plot confidence signal ERP over time (use betas rather than a weights)
    across_epochs_erp = epochs.average().get_data() # gives matrix with channels x timepoints (averaging across epochs)
    weighted_erp = across_epochs_erp * betas.reshape((betas.size, 1)) # weigh the voltages by the channel weights
    average_weighted_erp = weighted_erp.mean(axis=0) # compute the average across electrodes 
    x_axis = np.arange(epoch_start, epoch_end, (epoch_end-epoch_start)/(n_timepoints))
    plt.plot(x_axis, average_weighted_erp)
    plt.show()
    print("Average voltages (across epochs) weighted by regression betas for each electrode \n\n")

    
    
    ######################################################################
    ####################### TRAINING ON EVEN TRIALS ###################### 
    ######################################################################
    
    print('\033[1m' + "\n\n ______ train on even trials ______\n")
    print('\033[0m')
    
    X_train_trials = X_even_trials
    X_train_per_timepoint = X_even_per_timepoint
    
    X_test_trials = X_odd_trials
    X_test_per_timepoint = X_odd_per_timepoint
    
    
    y_train_trials = y_even_trials
    y_train_per_timepoint = y_even_per_timepoint
    
    y_test_trials = y_odd_trials
    y_test_per_timepoint = y_odd_per_timepoint
    
    
    
    # train classifier
    regressor = LinearRegression()  
    regressor.fit(X_train_per_timepoint, y_train_per_timepoint) #training the algorithm on every timepoint
    betas = regressor.coef_
    intercept = regressor.intercept_
    
    # predict y from X_test 
    # use predict function on individual timepoints
    y_pred_per_timepoint = regressor.predict(X_test_per_timepoint) 
    # then average predicted values within an epoch
    epoch_y_pred = np.mean(y_pred_per_timepoint.reshape(-1, n_timepoints), axis=1)

    print('\033[1m' + "\n test data")
    print('\033[0m')
    # R squared for timepoints
    print('R-squared timepoint data:', r2_score(y_test_per_timepoint, y_pred_per_timepoint))
#     # sanity check - manual computation gives same R-squared
#     actual_minus_predicted = sum((y_test_per_timepoint - y_pred_per_timepoint_1)**2)
#     actual_minus_actual_mean = sum((y_test_per_timepoint - y_test_per_timepoint.mean())**2)
#     r2 = 1 - actual_minus_predicted/actual_minus_actual_mean
#     print('R-squared timepoint data:', r2)
    
    # R squared for trials
    print('R-squared trial data:', r2_score(y_test_trials, epoch_y_pred))
#     # sanity check - manual computation gives same R-squared
#     actual_minus_predicted = sum((y_test_trials - epoch_y_pred_1)**2)
#     actual_minus_actual_mean = sum((y_test_trials - y_test_trials.mean())**2)
#     r2 = 1 - actual_minus_predicted/actual_minus_actual_mean
#     print('R-squared trial data:', r2)
    # plot y against predicted y
    sns.regplot(x=y_test_trials, y=epoch_y_pred, scatter_kws={'alpha':0.5})
    plt.xlabel('Reported confidence')
    plt.ylabel('Classifier prediction')
    plt.title('Reported trial confidence against predicted trial confidence test data')
    plt.show();
    

    # R squared for training data
    print('\033[1m' + "\n train data")
    print('\033[0m')
    # R squared for timepoints
    y_pred_per_timepoint_training = regressor.predict(X_train_per_timepoint) 
    print('R-squared training timepoint data:', r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))
    # R squared for trials
    epoch_y_pred_train = np.mean(y_pred_per_timepoint_training.reshape(-1, n_timepoints), axis=1)
    print('R-squared training trial data:', r2_score(y_train_trials, epoch_y_pred_train))
    # plot y against predicted y
    sns.regplot(x=y_train_trials, y=epoch_y_pred_train, scatter_kws={'alpha':0.5})
    plt.xlabel('Reported confidence')
    plt.ylabel('Classifier prediction')
    plt.title('Reported trial confidence against predicted trial confidence train data')
    plt.show();
    
    all_scores_test.append(r2_score(y_test_per_timepoint, y_pred_per_timepoint))
    all_scores_test_trialwise.append(r2_score(y_test_trials, epoch_y_pred))
    all_scores_train.append(r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))
    all_scores_train_trialwise.append(r2_score(y_train_trials, epoch_y_pred_train))
    

    ####################### PLOTTING ####################### 
    
    # plot topoplot with beta weights
    mne.viz.plot_topomap(data=betas, pos=epochs.info)
    print("Beta weights for each channel \n\n")
    
    # plot topoplot with sensor projections (see Parra et al paper for equation)
    y = y_pred_per_timepoint
    x = X_test_per_timepoint
    a =  y.T@x * ( 1 / (y.T@y) )
    mne.viz.plot_topomap(data=a, pos=epochs.info)
    print("Sensor projections \n\n")

#     # shuffle ys --> gives same topoplot because my predictions are basically random and then this just plots the average voltage which happens to look like P3
#     y = y_pred_per_timepoint
#     np.random.shuffle(y)
#     x = X_test_per_timepoint
#     a =  y.T@x * ( 1 / (y.T@y) )
#     mne.viz.plot_topomap(data=a, pos=epochs.info)
#     print("Sensor projections shuffled ys \n\n")
    
#     # shuffle xs --> gives random topoplot
#     y = y_pred_per_timepoint
#     x = X_test_per_timepoint
#     def shuffle_along_axis(a, axis):
#         idx = np.random.rand(*a.shape).argsort(axis=axis)
#         return np.take_along_axis(a,idx,axis=axis)
#     x = shuffle_along_axis(x, 1)
#     a =  y.T@x * ( 1 / (y.T@y) )
#     mne.viz.plot_topomap(data=a, pos=epochs.info)
#     print("Sensor projections shuffled xs \n\n")

    # plot confidence signal ERP over time (use betas rather than a weights)
    across_epochs_erp = epochs.average().get_data() # gives matrix with channels x timepoints (averaging across epochs)
    weighted_erp = across_epochs_erp * betas.reshape((betas.size, 1)) # weigh the voltages by the channel weights
    average_weighted_erp = weighted_erp.mean(axis=0) # compute the average across electrodes 
    x_axis = np.arange(epoch_start, epoch_end, (epoch_end-epoch_start)/(n_timepoints))
    plt.plot(x_axis, average_weighted_erp)
    plt.show()
    print("Average voltages (across epochs) weighted by regression betas for each electrode \n\n")    

    



R-squared timepoint wise

In [ ]:

print('\n Average R-squared test timepoint-wise: %f' % (sum(all_scores_test) / len(all_scores_test),))



In [ ]:

print('\n Average R-squared train timepoint-wise: %f' % (sum(all_scores_train) / len(all_scores_train),))



R-squared trial wise

In [ ]:

print('\n Average R-squared test trialwise: %f' % (sum(all_scores_test_trialwise) / len(all_scores_test_trialwise),))



In [ ]:

print('\n Average R-squared train trialwise: %f' % (sum(all_scores_train_trialwise) / len(all_scores_train_trialwise),))



### Add partner condition as predictor

In [ ]:
all_scores_test = []
all_scores_test_trialwise = []
all_scores_train = []
all_scores_train_trialwise = []


for subject in participant_numbers:
    print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % subject)
    print('\033[0m')
    
    
    # merge response epochs from both sessions for the participant
    participant_files = []
    for session in sessions:
        participant_files.append('%i_%i' % (subject, session))
    epochs = load_all_eeg(path='%s/' % input_dir, files=participant_files)
    
    # crop epochs
    epochs = epochs.crop(tmin=epoch_start, tmax=epoch_end, include_tmax=True)
    
    
    ######################################################################
    ####################### INDEPENDENT VARIABLE ######################### 
    ######################################################################
    
    # get epoched data as a numpy array of shape (n_epochs, n_channels, n_times) (600, 64, 127) (250 Hz --> every 4 ms)
    X = epochs.get_data()
    n_epochs = X.shape[0]
    n_channels = X.shape[1]
    n_timepoints = X.shape[2]
    X = X.swapaxes(1, 2) # (n_epochs, n_times, n_channels) (600, 127, 64)
    
    # divide into odd and even trials (3D array) (300, 127, 64)
    X_odd_trials = X[1::2, :, :] # select every second epoch (start:stop:step, :, :)
    X_even_trials = X[0::2, :, :] # select every other second epoch
    
    # collapse across trials and timepoints (2D array) (triple checked this and it is correctly sorted as e1t1, e1t2, e1t3, ...)
    X_odd_per_timepoint = X_odd_trials.reshape(-1, n_channels) # (n_epochs x n_times, n_channels) --> n_samples, n_features
    X_even_per_timepoint = X_even_trials.reshape(-1, n_channels) # (n_epochs x n_times, n_channels) --> n_samples, n_features

    
    # get partner for every trial as a numeric predictor
    partner = epochs.metadata['partner'].to_numpy()
    partner[partner=='underconfident']= -1
    partner[partner=='overconfident']= 1

    # divide into odd and even trials
    partner_odd_trials = partner[1::2]
    partner_even_trials = partner[0::2]

    # make from (n_epochs) to (n_epochs x n_timepoints, ) where partner stays the same
    partner_odd_per_timepoint = np.repeat(partner_odd_trials, n_timepoints)
    partner_even_per_timepoint = np.repeat(partner_even_trials, n_timepoints)
    
    # add partner as a predictor variable
    X_odd_per_timepoint = np.column_stack([X_odd_per_timepoint,partner_odd_per_timepoint])
    X_even_per_timepoint = np.column_stack([X_even_per_timepoint,partner_even_per_timepoint])

    ######################################################################
    ######################## DEPENDENT VARIABLE ########################## 
    ######################################################################
    
    # dependent variable as numpy array (n_epochs,)
    y = epochs.metadata['participant_confidence'].to_numpy()
    
    # divide into odd and even trials
    y_odd_trials = y[1::2]
    y_even_trials = y[0::2]
    
    # make from (n_epochs) to (n_epochs x n_timepoints, ) where confidence value stays the same
    y_odd_per_timepoint = np.repeat(y_odd_trials, n_timepoints)
    y_even_per_timepoint = np.repeat(y_even_trials, n_timepoints)
     
    
    ######################################################################
    ####################### TRAINING ON ODD TRIALS ####################### 
    ######################################################################
    
    print('\033[1m' + "\n\n ______ train on odd trials ______\n")
    print('\033[0m')
    
    X_train_trials = X_odd_trials
    X_train_per_timepoint = X_odd_per_timepoint
    
    X_test_trials = X_even_trials
    X_test_per_timepoint = X_even_per_timepoint
    
    
    y_train_trials = y_odd_trials
    y_train_per_timepoint = y_odd_per_timepoint
    
    y_test_trials = y_even_trials
    y_test_per_timepoint = y_even_per_timepoint
    
    
    
    # train classifier
    regressor = LinearRegression()  
    regressor.fit(X_train_per_timepoint, y_train_per_timepoint) #training the algorithm on every timepoint
    betas = regressor.coef_
        
    intercept = regressor.intercept_
    
    # predict y from X_test 
    # use predict function on individual timepoints
    y_pred_per_timepoint = regressor.predict(X_test_per_timepoint) 
    # then average predicted values within an epoch
    epoch_y_pred = np.mean(y_pred_per_timepoint.reshape(-1, n_timepoints), axis=1)
    

    print('\033[1m' + "\n test data")
    print('\033[0m')
    # R squared for timepoints
    print('R-squared timepoint data:', r2_score(y_test_per_timepoint, y_pred_per_timepoint))
    
    # R squared for trials
    print('R-squared trial data:', r2_score(y_test_trials, epoch_y_pred))
    sns.regplot(x=y_test_trials, y=epoch_y_pred, scatter_kws={'alpha':0.5})
    plt.xlabel('Reported confidence')
    plt.ylabel('Classifier prediction')
    plt.title('Reported trial confidence against predicted trial confidence test data')
    plt.show();
    

    # R squared for training data
    print('\033[1m' + "\n train data")
    print('\033[0m')
    # R squared for timepoints
    y_pred_per_timepoint_training = regressor.predict(X_train_per_timepoint) 
    print('R-squared training timepoint data:', r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))
    # R squared for trials
    epoch_y_pred_train = np.mean(y_pred_per_timepoint_training.reshape(-1, n_timepoints), axis=1)
    print('R-squared training trial data:', r2_score(y_train_trials, epoch_y_pred_train))
    # plot y against predicted y
    sns.regplot(x=y_train_trials, y=epoch_y_pred_train, scatter_kws={'alpha':0.5})
    plt.xlabel('Reported confidence')
    plt.ylabel('Classifier prediction')
    plt.title('Reported trial confidence against predicted trial confidence train data')
    plt.show();
    
    all_scores_test.append(r2_score(y_test_per_timepoint, y_pred_per_timepoint))
    all_scores_test_trialwise.append(r2_score(y_test_trials, epoch_y_pred))
    all_scores_train.append(r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))
    all_scores_train_trialwise.append(r2_score(y_train_trials, epoch_y_pred_train))
    

    ####################### PLOTTING ####################### 
    
    # plot topoplot with beta weights minus partner beta
    betas = betas[:-1]
    mne.viz.plot_topomap(data=betas, pos=epochs.info)
    print("Beta weights for each channel \n\n")

    # plot topoplot with sensor projections (see Parra et al paper for equation)
    y = y_pred_per_timepoint
    x = X_test_per_timepoint[:, :-1]
    a =  y.T@x * ( 1 / (y.T@y) )
    mne.viz.plot_topomap(data=a, pos=epochs.info)
    print("Sensor projections \n\n")
    
    # plot confidence signal ERP over time (use betas rather than a weights)
    across_epochs_erp = epochs.average().get_data() # gives matrix with channels x timepoints (averaging across epochs)
    weighted_erp = across_epochs_erp * betas.reshape((betas.size, 1)) # weigh the voltages by the channel weights
    average_weighted_erp = weighted_erp.mean(axis=0) # compute the average across electrodes 
    x_axis = np.arange(epoch_start, epoch_end, (epoch_end-epoch_start)/(n_timepoints))
    plt.plot(x_axis, average_weighted_erp)
    plt.show()
    print("Average voltages (across epochs) weighted by regression betas for each electrode \n\n")

    
    
    ######################################################################
    ####################### TRAINING ON EVEN TRIALS ###################### 
    ######################################################################
    
    print('\033[1m' + "\n\n ______ train on even trials ______\n")
    print('\033[0m')
    
    X_train_trials = X_even_trials
    X_train_per_timepoint = X_even_per_timepoint
    
    X_test_trials = X_odd_trials
    X_test_per_timepoint = X_odd_per_timepoint
    
    
    y_train_trials = y_even_trials
    y_train_per_timepoint = y_even_per_timepoint
    
    y_test_trials = y_odd_trials
    y_test_per_timepoint = y_odd_per_timepoint
    
    
    
    # train classifier
    regressor = LinearRegression()  
    regressor.fit(X_train_per_timepoint, y_train_per_timepoint) #training the algorithm on every timepoint
    betas = regressor.coef_
    intercept = regressor.intercept_
    
    # predict y from X_test 
    # use predict function on individual timepoints
    y_pred_per_timepoint = regressor.predict(X_test_per_timepoint) 
    # then average predicted values within an epoch
    epoch_y_pred = np.mean(y_pred_per_timepoint.reshape(-1, n_timepoints), axis=1)

    print('\033[1m' + "\n test data")
    print('\033[0m')
    # R squared for timepoints
    print('R-squared timepoint data:', r2_score(y_test_per_timepoint, y_pred_per_timepoint))
    
    # R squared for trials
    print('R-squared trial data:', r2_score(y_test_trials, epoch_y_pred))
    sns.regplot(x=y_test_trials, y=epoch_y_pred, scatter_kws={'alpha':0.5})
    plt.xlabel('Reported confidence')
    plt.ylabel('Classifier prediction')
    plt.title('Reported trial confidence against predicted trial confidence test data')
    plt.show();
    

    # R squared for training data
    print('\033[1m' + "\n train data")
    print('\033[0m')
    # R squared for timepoints
    y_pred_per_timepoint_training = regressor.predict(X_train_per_timepoint) 
    print('R-squared training timepoint data:', r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))
    # R squared for trials
    epoch_y_pred_train = np.mean(y_pred_per_timepoint_training.reshape(-1, n_timepoints), axis=1)
    print('R-squared training trial data:', r2_score(y_train_trials, epoch_y_pred_train))
    # plot y against predicted y
    sns.regplot(x=y_train_trials, y=epoch_y_pred_train, scatter_kws={'alpha':0.5})
    plt.xlabel('Reported confidence')
    plt.ylabel('Classifier prediction')
    plt.title('Reported trial confidence against predicted trial confidence train data')
    plt.show();
    
    all_scores_test.append(r2_score(y_test_per_timepoint, y_pred_per_timepoint))
    all_scores_test_trialwise.append(r2_score(y_test_trials, epoch_y_pred))
    all_scores_train.append(r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))
    all_scores_train_trialwise.append(r2_score(y_train_trials, epoch_y_pred_train))
    

    ####################### PLOTTING ####################### 

    # plot topoplot with beta weights minus partner beta
    betas = betas[:-1]
    mne.viz.plot_topomap(data=betas , pos=epochs.info)
    print("Beta weights for each channel \n\n")
    
    # plot topoplot with sensor projections (see Parra et al paper for equation) minus partner parameter
    y = y_pred_per_timepoint
    x = X_test_per_timepoint[:, :-1]
    a =  y.T@x * ( 1 / (y.T@y) )
    mne.viz.plot_topomap(data=a, pos=epochs.info)
    print("Sensor projections \n\n")

    # plot confidence signal ERP over time (use betas rather than a weights)
    across_epochs_erp = epochs.average().get_data() # gives matrix with channels x timepoints (averaging across epochs)
    weighted_erp = across_epochs_erp * betas.reshape((betas.size, 1)) # weigh the voltages by the channel weights
    average_weighted_erp = weighted_erp.mean(axis=0) # compute the average across electrodes 
    x_axis = np.arange(epoch_start, epoch_end, (epoch_end-epoch_start)/(n_timepoints))
    plt.plot(x_axis, average_weighted_erp)
    plt.show()
    print("Average voltages (across epochs) weighted by regression betas for each electrode \n\n")    





In [ ]:
print('\n Average R-squared test timepoint-wise: %f' % (sum(all_scores_test) / len(all_scores_test),))


In [ ]:
print('\n Average R-squared train timepoint-wise: %f' % (sum(all_scores_train) / len(all_scores_train),))


In [ ]:
print('\n Average R-squared test trialwise: %f' % (sum(all_scores_test_trialwise) / len(all_scores_test_trialwise),))


In [ ]:
print('\n Average R-squared train trialwise: %f' % (sum(all_scores_train_trialwise) / len(all_scores_train_trialwise),))


### Add partner and task condition as predictors

In [ ]:
all_scores_test = []
all_scores_test_trialwise = []
all_scores_train = []
all_scores_train_trialwise = []


for subject in participant_numbers:
    print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % subject)
    print('\033[0m')
    
    
    # merge response epochs from both sessions for the participant
    participant_files = []
    for session in sessions:
        participant_files.append('%i_%i' % (subject, session))
    epochs = load_all_eeg(path='%s/' % input_dir, files=participant_files)
    
    # crop epochs
    epochs = epochs.crop(tmin=epoch_start, tmax=epoch_end, include_tmax=True)
    
    
    ######################################################################
    ####################### INDEPENDENT VARIABLE ######################### 
    ######################################################################
    
    # get epoched data as a numpy array of shape (n_epochs, n_channels, n_times) (600, 64, 127) (250 Hz --> every 4 ms)
    X = epochs.get_data()
    n_epochs = X.shape[0]
    n_channels = X.shape[1]
    n_timepoints = X.shape[2]
    X = X.swapaxes(1, 2) # (n_epochs, n_times, n_channels) (600, 127, 64)
    
    # divide into odd and even trials (3D array) (300, 127, 64)
    X_odd_trials = X[1::2, :, :] # select every second epoch (start:stop:step, :, :)
    X_even_trials = X[0::2, :, :] # select every other second epoch
    
    # collapse across trials and timepoints (2D array) (triple checked this and it is correctly sorted as e1t1, e1t2, e1t3, ...)
    X_odd_per_timepoint = X_odd_trials.reshape(-1, n_channels) # (n_epochs x n_times, n_channels) --> n_samples, n_features
    X_even_per_timepoint = X_even_trials.reshape(-1, n_channels) # (n_epochs x n_times, n_channels) --> n_samples, n_features

    
    # get partner for every trial as a numeric predictor
    partner = epochs.metadata['partner'].to_numpy()
    partner[partner=='underconfident']= -1
    partner[partner=='overconfident']= 1

    # divide into odd and even trials
    partner_odd_trials = partner[1::2]
    partner_even_trials = partner[0::2]

    # make from (n_epochs) to (n_epochs x n_timepoints, ) where partner stays the same
    partner_odd_per_timepoint = np.repeat(partner_odd_trials, n_timepoints)
    partner_even_per_timepoint = np.repeat(partner_even_trials, n_timepoints)
    
    # add partner as a predictor variable
    X_odd_per_timepoint = np.column_stack([X_odd_per_timepoint,partner_odd_per_timepoint])
    X_even_per_timepoint = np.column_stack([X_even_per_timepoint,partner_even_per_timepoint])
    
    
    # get condition for every trial as a numeric predictor
    condition = epochs.metadata['condition'].to_numpy()
    condition[condition=='ns']= -1
    condition[condition=='s']= 1

    # divide into odd and even trials
    condition_odd_trials = condition[1::2]
    condition_even_trials = condition[0::2]

    # make from (n_epochs) to (n_epochs x n_timepoints, ) where partner stays the same
    condition_odd_per_timepoint = np.repeat(condition_odd_trials, n_timepoints)
    condition_even_per_timepoint = np.repeat(condition_even_trials, n_timepoints)
    
    # add condition as a predictor variable
    X_odd_per_timepoint = np.column_stack([X_odd_per_timepoint,condition_odd_per_timepoint])
    X_even_per_timepoint = np.column_stack([X_even_per_timepoint,condition_even_per_timepoint])

    ######################################################################
    ######################## DEPENDENT VARIABLE ########################## 
    ######################################################################
    
    # dependent variable as numpy array (n_epochs,)
    y = epochs.metadata['participant_confidence'].to_numpy()
    
    # divide into odd and even trials
    y_odd_trials = y[1::2]
    y_even_trials = y[0::2]
    
    # make from (n_epochs) to (n_epochs x n_timepoints, ) where confidence value stays the same
    y_odd_per_timepoint = np.repeat(y_odd_trials, n_timepoints)
    y_even_per_timepoint = np.repeat(y_even_trials, n_timepoints)
     
    
    ######################################################################
    ####################### TRAINING ON ODD TRIALS ####################### 
    ######################################################################
    
    print('\033[1m' + "\n\n ______ train on odd trials ______\n")
    print('\033[0m')
    
    X_train_trials = X_odd_trials
    X_train_per_timepoint = X_odd_per_timepoint
    
    X_test_trials = X_even_trials
    X_test_per_timepoint = X_even_per_timepoint
    
    
    y_train_trials = y_odd_trials
    y_train_per_timepoint = y_odd_per_timepoint
    
    y_test_trials = y_even_trials
    y_test_per_timepoint = y_even_per_timepoint
    
    
    
    # train classifier
    regressor = LinearRegression()  
    regressor.fit(X_train_per_timepoint, y_train_per_timepoint) #training the algorithm on every timepoint
    betas = regressor.coef_
    intercept = regressor.intercept_
    
    # predict y from X_test 
#     # use predict function on individual timepoints
#     y_pred_per_timepoint = regressor.predict(X_test_per_timepoint) 
#     # then average predicted values within an epoch
#     epoch_y_pred = np.mean(y_pred_per_timepoint.reshape(-1, n_timepoints), axis=1)
    # use betas and intercept on individual timepoints   
    y_pred_per_timepoint = np.sum(X_test_per_timepoint * betas, axis=1) + intercept 
    # then average predicted values within an epoch
    epoch_y_pred = np.mean(y_pred_per_timepoint.reshape(-1, n_timepoints), axis=1)
    

    print('\033[1m' + "\n test data")
    print('\033[0m')
    # R squared for timepoints
    print('R-squared timepoint data:', r2_score(y_test_per_timepoint, y_pred_per_timepoint))
    
    # R squared for trials
    print('R-squared trial data:', r2_score(y_test_trials, epoch_y_pred))
#     sns.regplot(x=y_test_trials, y=epoch_y_pred, scatter_kws={'alpha':0.5})
#     plt.xlabel('Reported confidence')
#     plt.ylabel('Classifier prediction')
#     plt.title('Reported trial confidence against predicted trial confidence test data')
#     plt.show();
    

    # R squared for training data
    print('\033[1m' + "\n train data")
    print('\033[0m')
    # R squared for timepoints
    y_pred_per_timepoint_training = regressor.predict(X_train_per_timepoint) 
    print('R-squared training timepoint data:', r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))
    # R squared for trials
    epoch_y_pred_train = np.mean(y_pred_per_timepoint_training.reshape(-1, n_timepoints), axis=1)
    print('R-squared training trial data:', r2_score(y_train_trials, epoch_y_pred_train))
    # plot y against predicted y
#     sns.regplot(x=y_train_trials, y=epoch_y_pred_train, scatter_kws={'alpha':0.5})
#     plt.xlabel('Reported confidence')
#     plt.ylabel('Classifier prediction')
#     plt.title('Reported trial confidence against predicted trial confidence train data')
#     plt.show();
    
    all_scores_test.append(r2_score(y_test_per_timepoint, y_pred_per_timepoint))
    all_scores_test_trialwise.append(r2_score(y_test_trials, epoch_y_pred))
    all_scores_train.append(r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))
    all_scores_train_trialwise.append(r2_score(y_train_trials, epoch_y_pred_train))
    

    ####################### PLOTTING ####################### 
    
    # plot topoplot with beta weights minus partner and condition betas
    betas = betas[:-2]
    mne.viz.plot_topomap(data=betas, pos=epochs.info)
    print("Beta weights for each channel \n\n")

    # plot topoplot with sensor projections (see Parra et al paper for equation) minus partner and condition betas
    y = y_pred_per_timepoint
    x = X_test_per_timepoint[:, :-2]
    a =  y.T@x * ( 1 / (y.T@y) )
    mne.viz.plot_topomap(data=a, pos=epochs.info)
    print("Sensor projections \n\n")
    
    # plot confidence signal ERP over time (use betas rather than a weights)
    across_epochs_erp = epochs.average().get_data() # gives matrix with channels x timepoints (averaging across epochs)
    weighted_erp = across_epochs_erp * betas.reshape((betas.size, 1)) # weigh the voltages by the channel weights
    average_weighted_erp = weighted_erp.mean(axis=0) # compute the average across electrodes 
    x_axis = np.arange(epoch_start, epoch_end, (epoch_end-epoch_start)/(n_timepoints))
    plt.plot(x_axis, average_weighted_erp)
    plt.show()
    print("Average voltages (across epochs) weighted by regression betas for each electrode \n\n")

    
    
    ######################################################################
    ####################### TRAINING ON EVEN TRIALS ###################### 
    ######################################################################
    
    print('\033[1m' + "\n\n ______ train on even trials ______\n")
    print('\033[0m')
    
    X_train_trials = X_even_trials
    X_train_per_timepoint = X_even_per_timepoint
    
    X_test_trials = X_odd_trials
    X_test_per_timepoint = X_odd_per_timepoint
    
    
    y_train_trials = y_even_trials
    y_train_per_timepoint = y_even_per_timepoint
    
    y_test_trials = y_odd_trials
    y_test_per_timepoint = y_odd_per_timepoint
    
    
    
    # train classifier
    regressor = LinearRegression()  
    regressor.fit(X_train_per_timepoint, y_train_per_timepoint) #training the algorithm on every timepoint
    betas = regressor.coef_
    intercept = regressor.intercept_
    
    # predict y from X_test 
#     # use predict function on individual timepoints
#     y_pred_per_timepoint = regressor.predict(X_test_per_timepoint) 
#     # then average predicted values within an epoch
#     epoch_y_pred = np.mean(y_pred_per_timepoint.reshape(-1, n_timepoints), axis=1)
    # use betas and intercept on individual timepoints   
    y_pred_per_timepoint = np.sum(X_test_per_timepoint * betas, axis=1) + intercept 
    # then average predicted values within an epoch
    epoch_y_pred = np.mean(y_pred_per_timepoint.reshape(-1, n_timepoints), axis=1)

    print('\033[1m' + "\n test data")
    print('\033[0m')
    # R squared for timepoints
    print('R-squared timepoint data:', r2_score(y_test_per_timepoint, y_pred_per_timepoint))
    
    # R squared for trials
    print('R-squared trial data:', r2_score(y_test_trials, epoch_y_pred))
#     sns.regplot(x=y_test_trials, y=epoch_y_pred, scatter_kws={'alpha':0.5})
#     plt.xlabel('Reported confidence')
#     plt.ylabel('Classifier prediction')
#     plt.title('Reported trial confidence against predicted trial confidence test data')
#     plt.show();
    

    # R squared for training data
    print('\033[1m' + "\n train data")
    print('\033[0m')
    # R squared for timepoints
    y_pred_per_timepoint_training = regressor.predict(X_train_per_timepoint) 
    print('R-squared training timepoint data:', r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))
    # R squared for trials
    epoch_y_pred_train = np.mean(y_pred_per_timepoint_training.reshape(-1, n_timepoints), axis=1)
    print('R-squared training trial data:', r2_score(y_train_trials, epoch_y_pred_train))
    # plot y against predicted y
#     sns.regplot(x=y_train_trials, y=epoch_y_pred_train, scatter_kws={'alpha':0.5})
#     plt.xlabel('Reported confidence')
#     plt.ylabel('Classifier prediction')
#     plt.title('Reported trial confidence against predicted trial confidence train data')
#     plt.show();
    
    all_scores_test.append(r2_score(y_test_per_timepoint, y_pred_per_timepoint))
    all_scores_test_trialwise.append(r2_score(y_test_trials, epoch_y_pred))
    all_scores_train.append(r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))
    all_scores_train_trialwise.append(r2_score(y_train_trials, epoch_y_pred_train))
    

    ####################### PLOTTING ####################### 

    # plot topoplot with beta weights minus partner and condition betas
    betas = betas[:-2]
    mne.viz.plot_topomap(data=betas , pos=epochs.info)
    print("Beta weights for each channel \n\n")
    
    # plot topoplot with sensor projections (see Parra et al paper for equation) minus partner and condition parameters
    y = y_pred_per_timepoint
    x = X_test_per_timepoint[:, :-2]
    a =  y.T@x * ( 1 / (y.T@y) )
    mne.viz.plot_topomap(data=a, pos=epochs.info)
    print("Sensor projections \n\n")

    # plot confidence signal ERP over time (use betas rather than a weights)
    across_epochs_erp = epochs.average().get_data() # gives matrix with channels x timepoints (averaging across epochs)
    weighted_erp = across_epochs_erp * betas.reshape((betas.size, 1)) # weigh the voltages by the channel weights
    average_weighted_erp = weighted_erp.mean(axis=0) # compute the average across electrodes 
    x_axis = np.arange(epoch_start, epoch_end, (epoch_end-epoch_start)/(n_timepoints))
    plt.plot(x_axis, average_weighted_erp)
    plt.show()
    print("Average voltages (across epochs) weighted by regression betas for each electrode \n\n")    





In [ ]:
print('\n Average R-squared test timepoint-wise: %f' % (sum(all_scores_test) / len(all_scores_test),))


In [ ]:
print('\n Average R-squared train timepoint-wise: %f' % (sum(all_scores_train) / len(all_scores_train),))


In [ ]:
print('\n Average R-squared test trialwise: %f' % (sum(all_scores_test_trialwise) / len(all_scores_test_trialwise),))


In [ ]:
print('\n Average R-squared train trialwise: %f' % (sum(all_scores_train_trialwise) / len(all_scores_train_trialwise),))


### Regression with only partner and task condition but without EEG

In [ ]:
all_scores_test = []
all_scores_train = []

for subject in participant_numbers:
    print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % subject)
    print('\033[0m')
    
    
    # merge response epochs from both sessions for the participant
    participant_files = []
    for session in sessions:
        participant_files.append('%i_%i' % (subject, session))
    epochs = load_all_eeg(path='%s/' % input_dir, files=participant_files)
    
    # crop epochs
    epochs = epochs.crop(tmin=epoch_start, tmax=epoch_end, include_tmax=True)
    
    
    ######################################################################
    ####################### INDEPENDENT VARIABLE ######################### 
    ######################################################################

    # get partner for every trial as a numeric predictor
    partner = epochs.metadata['partner'].to_numpy()
    partner[partner=='underconfident']= -1
    partner[partner=='overconfident']= 1

    # divide into odd and even trials
    partner_odd_trials = partner[1::2]
    partner_even_trials = partner[0::2]
    
    # get condition for every trial as a numeric predictor
    condition = epochs.metadata['condition'].to_numpy()
    condition[condition=='ns']= -1
    condition[condition=='s']= 1

    # divide into odd and even trials
    condition_odd_trials = condition[1::2]
    condition_even_trials = condition[0::2]

    
    # add partner and condition as predictor variables
    X_odd_trials = np.column_stack([partner_odd_trials,condition_odd_trials])
    X_even_trials = np.column_stack([partner_even_trials,condition_even_trials])

    ######################################################################
    ######################## DEPENDENT VARIABLE ########################## 
    ######################################################################
    
    # dependent variable as numpy array (n_epochs,)
    y = epochs.metadata['participant_confidence'].to_numpy()
    
    # divide into odd and even trials
    y_odd_trials = y[1::2]
    y_even_trials = y[0::2]
     
    
    ######################################################################
    ####################### TRAINING ON ODD TRIALS ####################### 
    ######################################################################
    
    print('\033[1m' + "\n\n ______ train on odd trials ______\n")
    print('\033[0m')
    
    X_train_trials = X_odd_trials
    X_test_trials = X_even_trials
       
    y_train_trials = y_odd_trials 
    y_test_trials = y_even_trials
    
    
    # train classifier
    regressor = LinearRegression()  
    regressor.fit(X_train_trials, y_train_trials)
    betas = regressor.coef_
    intercept = regressor.intercept_
    
    # predict y from X_test 
    # use predict function on individual timepoints
    y_pred = regressor.predict(X_test_trials) 
    print('\033[1m' + "\n test data")
    print('\033[0m')
    # R squared for trials
    print('R-squared trial data:', r2_score(y_test_trials, y_pred))
    sns.regplot(x=y_test_trials, y=y_pred, scatter_kws={'alpha':0.5})
    plt.xlabel('Reported confidence')
    plt.ylabel('Classifier prediction')
    plt.title('Reported trial confidence against predicted trial confidence test data')
    plt.show();
    

    # R squared for training data
    print('\033[1m' + "\n train data")
    print('\033[0m')
    y_pred_train = regressor.predict(X_train_trials) 
    print('R-squared training trial data:', r2_score(y_train_trials, y_pred_train))
    # plot y against predicted y
    sns.regplot(x=y_train_trials, y=y_pred_train, scatter_kws={'alpha':0.5})
    plt.xlabel('Reported confidence')
    plt.ylabel('Classifier prediction')
    plt.title('Reported trial confidence against predicted trial confidence train data')
    plt.show();
    
    all_scores_test.append(r2_score(y_test_per_timepoint, y_pred_per_timepoint))
    all_scores_train.append(r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))

    
    
    ######################################################################
    ####################### TRAINING ON EVEN TRIALS ###################### 
    ######################################################################
    
    print('\033[1m' + "\n\n ______ train on even trials ______\n")
    print('\033[0m')
    
    X_train_trials = X_even_trials
    X_test_trials = X_odd_trials
       
    y_train_trials = y_even_trials 
    y_test_trials = y_odd_trials
    
    
    # train classifier
    regressor = LinearRegression()  
    regressor.fit(X_train_trials, y_train_trials)
    betas = regressor.coef_
    intercept = regressor.intercept_
    
    # predict y from X_test 
    # use predict function on individual timepoints
    y_pred = regressor.predict(X_test_trials) 

    print('\033[1m' + "\n test data")
    print('\033[0m')
    # R squared for trials
    print('R-squared trial data:', r2_score(y_test_trials, y_pred))
    sns.regplot(x=y_test_trials, y=y_pred, scatter_kws={'alpha':0.5})
    plt.xlabel('Reported confidence')
    plt.ylabel('Classifier prediction')
    plt.title('Reported trial confidence against predicted trial confidence test data')
    plt.show();
    

    # R squared for training data
    print('\033[1m' + "\n train data")
    print('\033[0m')
    y_pred_train = regressor.predict(X_train_trials) 
    print('R-squared training trial data:', r2_score(y_train_trials, y_pred_train))
    # plot y against predicted y
    sns.regplot(x=y_train_trials, y=y_pred_train, scatter_kws={'alpha':0.5})
    plt.xlabel('Reported confidence')
    plt.ylabel('Classifier prediction')
    plt.title('Reported trial confidence against predicted trial confidence train data')
    plt.show();
    
    all_scores_test.append(r2_score(y_test_per_timepoint, y_pred_per_timepoint))
    all_scores_train.append(r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))





In [ ]:
print('\n Average R-squared test: %f' % (sum(all_scores_test) / len(all_scores_test),))


In [ ]:
print('\n Average R-squared train: %f' % (sum(all_scores_train) / len(all_scores_train),))


### Regression with partner and task conditions as predictors for trainig the classifier but then only use EEG betas for the test data

In [ ]:
all_scores_test = []
all_scores_test_trialwise = []
all_scores_train = []
all_scores_train_trialwise = []


for subject in participant_numbers:
    print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % subject)
    print('\033[0m')
    
    
    # merge response epochs from both sessions for the participant
    participant_files = []
    for session in sessions:
        participant_files.append('%i_%i' % (subject, session))
    epochs = load_all_eeg(path='%s/' % input_dir, files=participant_files)
    
    # crop epochs
    epochs = epochs.crop(tmin=epoch_start, tmax=epoch_end, include_tmax=True)
    
    
    ######################################################################
    ####################### INDEPENDENT VARIABLE ######################### 
    ######################################################################
    
    # get epoched data as a numpy array of shape (n_epochs, n_channels, n_times) (600, 64, 127) (250 Hz --> every 4 ms)
    X = epochs.get_data()
    n_epochs = X.shape[0]
    n_channels = X.shape[1]
    n_timepoints = X.shape[2]
    X = X.swapaxes(1, 2) # (n_epochs, n_times, n_channels) (600, 127, 64)
    
    # divide into odd and even trials (3D array) (300, 127, 64)
    X_odd_trials = X[1::2, :, :] # select every second epoch (start:stop:step, :, :)
    X_even_trials = X[0::2, :, :] # select every other second epoch
    
    # collapse across trials and timepoints (2D array) (triple checked this and it is correctly sorted as e1t1, e1t2, e1t3, ...)
    X_odd_per_timepoint = X_odd_trials.reshape(-1, n_channels) # (n_epochs x n_times, n_channels) --> n_samples, n_features
    X_even_per_timepoint = X_even_trials.reshape(-1, n_channels) # (n_epochs x n_times, n_channels) --> n_samples, n_features

    
    # get partner for every trial as a numeric predictor
    partner = epochs.metadata['partner'].to_numpy()
    partner[partner=='underconfident']= -1
    partner[partner=='overconfident']= 1

    # divide into odd and even trials
    partner_odd_trials = partner[1::2]
    partner_even_trials = partner[0::2]

    # make from (n_epochs) to (n_epochs x n_timepoints, ) where partner stays the same
    partner_odd_per_timepoint = np.repeat(partner_odd_trials, n_timepoints)
    partner_even_per_timepoint = np.repeat(partner_even_trials, n_timepoints)
    
    # add partner as a predictor variable
    X_odd_per_timepoint = np.column_stack([X_odd_per_timepoint,partner_odd_per_timepoint])
    X_even_per_timepoint = np.column_stack([X_even_per_timepoint,partner_even_per_timepoint])
    
    
    # get condition for every trial as a numeric predictor
    condition = epochs.metadata['condition'].to_numpy()
    condition[condition=='ns']= -1
    condition[condition=='s']= 1

    # divide into odd and even trials
    condition_odd_trials = condition[1::2]
    condition_even_trials = condition[0::2]

    # make from (n_epochs) to (n_epochs x n_timepoints, ) where partner stays the same
    condition_odd_per_timepoint = np.repeat(condition_odd_trials, n_timepoints)
    condition_even_per_timepoint = np.repeat(condition_even_trials, n_timepoints)
    
    # add condition as a predictor variable
    X_odd_per_timepoint = np.column_stack([X_odd_per_timepoint,condition_odd_per_timepoint])
    X_even_per_timepoint = np.column_stack([X_even_per_timepoint,condition_even_per_timepoint])

    ######################################################################
    ######################## DEPENDENT VARIABLE ########################## 
    ######################################################################
    
    # dependent variable as numpy array (n_epochs,)
    y = epochs.metadata['participant_confidence'].to_numpy()
    
    # divide into odd and even trials
    y_odd_trials = y[1::2]
    y_even_trials = y[0::2]
    
    # make from (n_epochs) to (n_epochs x n_timepoints, ) where confidence value stays the same
    y_odd_per_timepoint = np.repeat(y_odd_trials, n_timepoints)
    y_even_per_timepoint = np.repeat(y_even_trials, n_timepoints)
     
    
    ######################################################################
    ####################### TRAINING ON ODD TRIALS ####################### 
    ######################################################################
    
    print('\033[1m' + "\n\n ______ train on odd trials ______\n")
    print('\033[0m')
    
    X_train_trials = X_odd_trials
    X_train_per_timepoint = X_odd_per_timepoint
    
    X_test_trials = X_even_trials
    X_test_per_timepoint = X_even_per_timepoint
    
    
    y_train_trials = y_odd_trials
    y_train_per_timepoint = y_odd_per_timepoint
    
    y_test_trials = y_even_trials
    y_test_per_timepoint = y_even_per_timepoint
    
    
    
    # train classifier
    regressor = LinearRegression()  
    regressor.fit(X_train_per_timepoint, y_train_per_timepoint) #training the algorithm on every timepoint
    betas = regressor.coef_
    intercept = regressor.intercept_
    
    # predict y from X_test without task and partner betas   
    # use betas and intercept on individual timepoints   
    y_pred_per_timepoint = np.sum(X_test_per_timepoint[:, :-2] * betas[:-2], axis=1) + intercept 
    # then average predicted values within an epoch
    epoch_y_pred = np.mean(y_pred_per_timepoint.reshape(-1, n_timepoints), axis=1)
    

    print('\033[1m' + "\n test data")
    print('\033[0m')
    # R squared for timepoints
    print('R-squared timepoint data:', r2_score(y_test_per_timepoint, y_pred_per_timepoint))
    
    # R squared for trials
    print('R-squared trial data:', r2_score(y_test_trials, epoch_y_pred))
    

    # R squared for training data
    print('\033[1m' + "\n train data")
    print('\033[0m')
    # R squared for timepoints
    y_pred_per_timepoint_training = regressor.predict(X_train_per_timepoint) 
    print('R-squared training timepoint data:', r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))
    # R squared for trials
    epoch_y_pred_train = np.mean(y_pred_per_timepoint_training.reshape(-1, n_timepoints), axis=1)
    print('R-squared training trial data:', r2_score(y_train_trials, epoch_y_pred_train))
    
    all_scores_test.append(r2_score(y_test_per_timepoint, y_pred_per_timepoint))
    all_scores_test_trialwise.append(r2_score(y_test_trials, epoch_y_pred))
    all_scores_train.append(r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))
    all_scores_train_trialwise.append(r2_score(y_train_trials, epoch_y_pred_train))

    
    
    ######################################################################
    ####################### TRAINING ON EVEN TRIALS ###################### 
    ######################################################################
    
    print('\033[1m' + "\n\n ______ train on even trials ______\n")
    print('\033[0m')
    
    X_train_trials = X_even_trials
    X_train_per_timepoint = X_even_per_timepoint
    
    X_test_trials = X_odd_trials
    X_test_per_timepoint = X_odd_per_timepoint
    
    
    y_train_trials = y_even_trials
    y_train_per_timepoint = y_even_per_timepoint
    
    y_test_trials = y_odd_trials
    y_test_per_timepoint = y_odd_per_timepoint
    
    
    
    # train classifier
    regressor = LinearRegression()  
    regressor.fit(X_train_per_timepoint, y_train_per_timepoint) #training the algorithm on every timepoint
    betas = regressor.coef_
    intercept = regressor.intercept_
    
    # predict y from X_test without task and partner betas   
    # use betas and intercept on individual timepoints   
    y_pred_per_timepoint = np.sum(X_test_per_timepoint[:, :-2] * betas[:-2], axis=1) + intercept 
    # then average predicted values within an epoch
    epoch_y_pred = np.mean(y_pred_per_timepoint.reshape(-1, n_timepoints), axis=1)

    print('\033[1m' + "\n test data")
    print('\033[0m')
    # R squared for timepoints
    print('R-squared timepoint data:', r2_score(y_test_per_timepoint, y_pred_per_timepoint))
    
    # R squared for trials
    print('R-squared trial data:', r2_score(y_test_trials, epoch_y_pred))
    

    # R squared for training data
    print('\033[1m' + "\n train data")
    print('\033[0m')
    # R squared for timepoints
    y_pred_per_timepoint_training = regressor.predict(X_train_per_timepoint) 
    print('R-squared training timepoint data:', r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))
    # R squared for trials
    epoch_y_pred_train = np.mean(y_pred_per_timepoint_training.reshape(-1, n_timepoints), axis=1)
    print('R-squared training trial data:', r2_score(y_train_trials, epoch_y_pred_train))
    
    all_scores_test.append(r2_score(y_test_per_timepoint, y_pred_per_timepoint))
    all_scores_test_trialwise.append(r2_score(y_test_trials, epoch_y_pred))
    all_scores_train.append(r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))
    all_scores_train_trialwise.append(r2_score(y_train_trials, epoch_y_pred_train)) 





In [ ]:
print('\n Average R-squared test timepoint-wise: %f' % (sum(all_scores_test) / len(all_scores_test),))


In [ ]:
print('\n Average R-squared train timepoint-wise: %f' % (sum(all_scores_train) / len(all_scores_train),))


In [ ]:
print('\n Average R-squared test trialwise: %f' % (sum(all_scores_test_trialwise) / len(all_scores_test_trialwise),))


In [ ]:
print('\n Average R-squared train trialwise: %f' % (sum(all_scores_train_trialwise) / len(all_scores_train_trialwise),))


## Ridge regression (regularization)

In [ ]:
%%capture

all_lr_scores_ridge = []

for subject in participant_numbers:   
#     print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % subject)
#     print('\033[0m')
    
    # merge response epochs from both sessions for the participant
    participant_files = []
    for session in sessions:
        participant_files.append('%i_%i' % (subject, session))
    epochs = load_all_eeg(path='%s/' % input_dir, files=participant_files)
    
    # crop epochs
    epochs = epochs.crop(tmin=epoch_start, tmax=epoch_end, include_tmax=True)
    
    # independent variables
    # get epoched data as a numpy array of shape (n_epochs, n_channels, n_times)
    X = epochs.get_data()
    
    n_epochs = X.shape[0]
    n_channels = X.shape[1]
    n_timepoints = X.shape[2]
    
    # collapse across trials and timepoints
    X = X.swapaxes(1, 2) # (n_epochs, n_times, n_channels)
    X = X.reshape(-1, n_channels) # (n_epochs x n_times, n_channels) --> n_samples, n_features

    # dependent variable as numpy array (n_epochs,)
    y = epochs.metadata['participant_confidence'].to_numpy()
    # extend from (n_epochs) to (n_epochs x n_timepoints, ) where confidence value stays the same
    y = np.repeat(y, n_timepoints)
    
    # fit regression
    reg = Ridge(alpha=.5).fit(X, y)
    score = reg.score(X, y)
#     print(score)
    
#     # regression betas
#     betas = reg.coef_
#     #print(betas)
    
#     mne.viz.plot_topomap(data=betas, pos=epochs.info)

    all_lr_scores_ridge.append(score)


In [ ]:

print('\n Average R-squared ridge regression: %f' % (sum(all_lr_scores_ridge) / len(all_lr_scores_ridge),))



## Logistic regression

In [ ]:
%%capture

all_lg_scores = []

for subject in participant_numbers:   
#     print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % subject)
#     print('\033[0m')
    
    # merge response epochs from both sessions for the participant
    participant_files = []
    for session in sessions:
        participant_files.append('%i_%i' % (subject, session))
    epochs = load_all_eeg(path='%s/' % input_dir, files=participant_files)
    
    # crop epochs
    epochs = epochs.crop(tmin=epoch_start, tmax=epoch_end, include_tmax=True)
    
    # independent variables
    # get epoched data as a numpy array of shape (n_epochs, n_channels, n_times)
    X = epochs.get_data()
    
    n_epochs = X.shape[0]
    n_channels = X.shape[1]
    n_timepoints = X.shape[2]
    
    # collapse across trials and timepoints
    X = X.swapaxes(1, 2) # (n_epochs, n_times, n_channels)
    X = X.reshape(-1, n_channels) # (n_epochs x n_times, n_channels) --> n_samples, n_features

    # dependent variable as numpy array (n_epochs,)
    y = epochs.metadata['participant_confidence'].to_numpy()
    # make it binary
    med_y = np.median(y)
    y[y<=med_y] = -1
    y[y>med_y] = 1
    
    # extend from (n_epochs) to (n_epochs x n_timepoints, ) where confidence value stays the same
    y = np.repeat(y, n_timepoints)
    
    # fit regression
    reg = LogisticRegression().fit(X, y)
    score = reg.score(X, y)
#     print(score)
    
#     # regression betas
#     betas = reg.coef_
#     #print(betas)
    
#     mne.viz.plot_topomap(data=betas, pos=epochs.info)

    all_lg_scores.append(score)


In [ ]:

print('Average accuracy logistic regression: %f' % (sum(all_lg_scores) / len(all_lg_scores),))


## Multinomial regression

confidence quintile split

In [ ]:
%%capture

all_mn_scores = []

for subject in participant_numbers:   
#     print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % subject)
#     print('\033[0m')
    
    # merge response epochs from both sessions for the participant
    participant_files = []
    for session in sessions:
        participant_files.append('%i_%i' % (subject, session))
    epochs = load_all_eeg(path='%s/' % input_dir, files=participant_files)
    
    # crop epochs
    epochs = epochs.crop(tmin=epoch_start, tmax=epoch_end, include_tmax=True)
    
    # independent variables
    # get epoched data as a numpy array of shape (n_epochs, n_channels, n_times)
    X = epochs.get_data()
    
    n_epochs = X.shape[0]
    n_channels = X.shape[1]
    n_timepoints = X.shape[2]
    
    # collapse across trials and timepoints
    X = X.swapaxes(1, 2) # (n_epochs, n_times, n_channels)
    X = X.reshape(-1, n_channels) # (n_epochs x n_times, n_channels) --> n_samples, n_features

    # dependent variable as numpy array (n_epochs,)
    y = epochs.metadata['participant_confidence'].to_numpy()
    # make it categorical
    quintiles = np.percentile( y, [20,40,60,80] )    
    y = 5 - (quintiles[:,None] >=  y).sum(0)
    
    # extend from (n_epochs) to (n_epochs x n_timepoints, ) where confidence value stays the same
    y = np.repeat(y, n_timepoints)
    
    # fit regression
    reg = LogisticRegression().fit(X, y)
    score = reg.score(X, y)
#     print(score)
    
#     # regression betas
#     betas = reg.coef_
#     #print(betas)
    
#     mne.viz.plot_topomap(data=betas, pos=epochs.info)

    all_mn_scores.append(score)


In [ ]:

print('Average accuracy multinomial regression using quintiles: %f' % (sum(all_mn_scores) / len(all_mn_scores),))


confidence quartile split

In [ ]:
%%capture

all_mn_scores = []

for subject in participant_numbers:   
#     print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % subject)
#     print('\033[0m')
    
    # merge response epochs from both sessions for the participant
    participant_files = []
    for session in sessions:
        participant_files.append('%i_%i' % (subject, session))
    epochs = load_all_eeg(path='%s/' % input_dir, files=participant_files)
    
    # crop epochs
    epochs = epochs.crop(tmin=epoch_start, tmax=epoch_end, include_tmax=True)
    
    # independent variables
    # get epoched data as a numpy array of shape (n_epochs, n_channels, n_times)
    X = epochs.get_data()
    
    n_epochs = X.shape[0]
    n_channels = X.shape[1]
    n_timepoints = X.shape[2]
    
    # collapse across trials and timepoints
    X = X.swapaxes(1, 2) # (n_epochs, n_times, n_channels)
    X = X.reshape(-1, n_channels) # (n_epochs x n_times, n_channels) --> n_samples, n_features

    # dependent variable as numpy array (n_epochs,)
    y = epochs.metadata['participant_confidence'].to_numpy()
    # make it categorical
    quartiles = np.percentile( y, [25,50,75] )    
    y = 4 - (quartiles[:,None] >=  y).sum(0)
    
    # extend from (n_epochs) to (n_epochs x n_timepoints, ) where confidence value stays the same
    y = np.repeat(y, n_timepoints)
    
    # fit regression
    reg = LogisticRegression().fit(X, y)
    score = reg.score(X, y)
#     print(score)
    
#     # regression betas
#     betas = reg.coef_
#     #print(betas)
    
#     mne.viz.plot_topomap(data=betas, pos=epochs.info)

    all_mn_scores.append(score)


In [ ]:

print('Average accuracy multinomial regression using quartiles: %f' % (sum(all_mn_scores) / len(all_mn_scores),))


## Correlations between confidence and indiviudal electrodes

plotting confidence median split

In [ ]:
participant_corrs = []

for subject in participant_numbers:   
    print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % subject)
    print('\033[0m')
    
    # merge response epochs from both sessions for the participant
    participant_files = []
    for session in sessions:
        participant_files.append('%i_%i' % (subject, session))
    epochs = load_all_eeg(path='%s/' % input_dir, files=participant_files)
    
    # crop epochs
    epochs = epochs.crop(tmin=epoch_start, tmax=epoch_end, include_tmax=True)
    
    # independent variables
    # get epoched data as a numpy array of shape (n_epochs, n_channels, n_times)
    # crop epochs for correlation computation
    X = epochs.get_data()
    
    # get epoch average for each channel (average across timepoints) (n_epochs, n_channels)
    X = np.mean(X, 2)
    # divide into even and odd trials
    X_even = X[0::2, :]
    X_odd = X[1::2, :]

    # dependent variable as numpy array (n_epochs,)
    y = epochs.metadata['participant_confidence'].to_numpy()
    # divide into odd an even trials
    y_even = y[0::2]
    y_odd = y[1::2]
    
    
    # compute correlation of every channel with confidence
    all_corrs = []
    for channel in range (X.shape[1]):
        x = X[:, channel]
        cor = np.corrcoef(y, x)[0, 1]
        all_corrs.append(cor)
    min_index = np.argmin(all_corrs)
    
    
    # plot topoploy of correlations
    print("Correlation of each channel with confidence \n\n")
    mne.viz.plot_topomap(data=all_corrs , pos=epochs.info, names=epochs.ch_names, show_names=True)
    
    
    print("\n Most negative correlation with confidence is at %s with %.3f" %(epochs.ch_names[min_index], max(all_corrs)))
#     sns.regplot(x=y, y=X[:, max_index], scatter_kws={'alpha':0.5})
#     plt.xlabel('Reported confidence')
#     plt.ylabel('Average voltage')
#     plt.show();
    
    participant_corrs.append(all_corrs)
    

    # plot high vs low confidence at the channel that min correlates with confidence
    print("\n high vs low confidence at the channel that most negatively correlates with confidence")
    median_confidence = epochs.metadata['participant_confidence'].median()
    low_conf_evoked = epochs['participant_confidence <= %i' % median_confidence].average()
    high_conf_evoked = epochs['participant_confidence > %i' % median_confidence].average()
    evokeds = dict(high_confidence=high_conf_evoked, low_confidence=low_conf_evoked)
    mne.viz.plot_compare_evokeds(evokeds, picks=[epochs.ch_names[min_index]], invert_y=False)
    
#     # plot confidence quartiles at the channel that max correlates with confidence
#     quartiles = np.percentile(y, [25,50,75]) 
#     y_quartiles = 4 - (quartiles[:,None] >=  y).sum(0)
#     epochs.metadata['confidence_quartile'] = y_quartiles
#     Q1_epochs = epochs['confidence_quartile == %i' % 1].average()
#     Q2_epochs = epochs['confidence_quartile == %i' % 2].average()
#     Q3_epochs = epochs['confidence_quartile == %i' % 3].average()
#     Q4_epochs = epochs['confidence_quartile == %i' % 4].average()
#     evokeds = dict(highest_confidence=Q4_epochs, high_confidence=Q3_epochs, low_confidence=Q2_epochs, lowest_confidence=Q1_epochs)
#     mne.viz.plot_compare_evokeds(evokeds, picks=[epochs.ch_names[max_index]], invert_y=False)
    
#     # downsampled plot
#     epochs = epochs.decimate(20)
#     Q1_epochs = epochs['confidence_quartile == %i' % 1].average()
#     Q2_epochs = epochs['confidence_quartile == %i' % 2].average()
#     Q3_epochs = epochs['confidence_quartile == %i' % 3].average()
#     Q4_epochs = epochs['confidence_quartile == %i' % 4].average()
#     evokeds = dict(highest_confidence=Q4_epochs, high_confidence=Q3_epochs, low_confidence=Q2_epochs, lowest_confidence=Q1_epochs)
#     mne.viz.plot_compare_evokeds(evokeds, picks=[epochs.ch_names[max_index]], invert_y=False)
    
    
    
#     # plot high vs low confidence z-scored by partner at the channel that max correlates with confidence
#     zscore = lambda x: (x - x.mean()) / x.std()
#     epochs.metadata['confidence_z_by_partner'] = epochs.metadata['participant_confidence'].groupby(epochs.metadata['partner']).transform(zscore)
#     median_confidence = epochs.metadata['confidence_z_by_partner'].median()
    
#     low_conf_epochs = epochs['confidence_z_by_partner <= %i' % median_confidence]
#     high_conf_epochs = epochs['confidence_z_by_partner > %i' % median_confidence]
#     low_conf_evoked = low_conf_epochs.average()
#     high_conf_evoked = high_conf_epochs.average()
#     evokeds = dict(high_confidence=high_conf_evoked, low_confidence=low_conf_evoked)
#     mne.viz.plot_compare_evokeds(evokeds, picks=[epochs.ch_names[max_index]], invert_y=False)
    
    
#     # compute correlation of every channel with confidence in even trials
#     all_corrs_even = []
#     for channel in range (X_even.shape[1]):
#         x_even = X_even[:, channel]
#         cor_even = np.corrcoef(y_even, x_even)[0, 1]
#         all_corrs_even.append(cor_even)
#     max_index_even = np.argmax(all_corrs_even)
#     print("\n\n For even trials, maximum correlation with confidence is at %s with %.3f" %(epochs.ch_names[max_index_even], max(all_corrs_even)))
#     cor_odd = np.correlate(y_odd, X_odd[:, max_index_even])
#     print("\n For odd trials, correlation with confidence at %s is %.3f" %(epochs.ch_names[max_index_even], cor_odd))

    
#     # compute correlation of every channel with confidence in odd trials
#     all_corrs_odd = []
#     for channel in range (X_odd.shape[1]):
#         x_odd = X_odd[:, channel]
#         cor_odd = np.corrcoef(y_odd, x_odd)[0, 1]
#         all_corrs_odd.append(cor_odd)
#     max_index_odd = np.argmax(all_corrs_odd)
#     print("\n\n For odd trials, maximum correlation with confidence is at %s with %.3f" %(epochs.ch_names[max_index_odd], max(all_corrs_odd)))
#     cor_even = np.correlate(y_even, X_even[:, max_index_odd])
#     print("\n For even trials, correlation with confidence at %s is %.3f" %(epochs.ch_names[max_index_odd], cor_even))


participant_correlations = np.array(participant_corrs)

In [ ]:
average_participant_correlations = np.mean(participant_correlations, 0)
average_participant_correlations

In [ ]:
np.min(average_participant_correlations)

In [ ]:
plt.rcParams['figure.figsize'] = [20, 10]
print("\n Average correlation of each channel with confidence across participants")
mne.viz.plot_topomap(data=average_participant_correlations , pos=epochs.info, names=epochs.ch_names, show_names=True)
import matplotlib.pyplot as plt

In [ ]:
largest_neg = epochs.ch_names[average_participant_correlations.argsort()[1]]
largest_neg

In [ ]:
second_largest_neg = epochs.ch_names[average_participant_correlations.argsort()[2]]
second_largest_neg

In [ ]:
third_largest_neg = epochs.ch_names[average_participant_correlations.argsort()[3]]
third_largest_neg

In [ ]:
fourth_largest_neg = epochs.ch_names[average_participant_correlations.argsort()[4]]
fourth_largest_neg

In [ ]:
fifth_largest_neg = epochs.ch_names[average_participant_correlations.argsort()[5]]
fifth_largest_neg

## Aggregate

In [ ]:
%%capture

participant_files = []

for session in sessions:
    for subject in participant_numbers:
        participant_files.append('%i_%i' % (subject, session))

remerge = False
if remerge:
    epochs = load_all_eeg(path='%s/' % input_dir, files=participant_files)
    epochs.save('mergedData/response_epoch_mastoids-epo.fif', overwrite=True)
else:
    epochs = mne.read_epochs('mergedData/response_epoch_mastoids-epo.fif')

data = epochs.metadata

In [ ]:
zscore = lambda x: (x - x.mean()) / x.std()
# zscore by participant
#epochs.metadata['confidence_z_by_participant'] = epochs.metadata['participant_confidence'].groupby(epochs.metadata['participant']).transform(zscore)


In [ ]:
strategic_epochs = epochs['condition == "s"']
strategic_epochs.metadata['confidence_z_by_participant'] = strategic_epochs.metadata['participant_confidence'].groupby(strategic_epochs.metadata['participant']).transform(zscore)

nonstrategic_epochs = epochs['condition == "ns"']
nonstrategic_epochs.metadata['confidence_z_by_participant'] = nonstrategic_epochs.metadata['participant_confidence'].groupby(nonstrategic_epochs.metadata['participant']).transform(zscore)


In [ ]:
roi = ["Pz", "CPz", "POz", "P1", "P2"]
#roi_1 = ["Pz", "CPz", "POz", "P1", "P2", "Cz"]
#roi_2 = ["P3", "CP3", "P1", "CP1", "Pz", "CPz", "P2", "CP2", "P4", "CP4"]
highest_corrs_neg = [largest_neg, second_largest_neg, third_largest_neg, fourth_largest_neg, fifth_largest_neg]

In [ ]:
## STRATEGIC

grand_average_underconf_high_conf = []
grand_average_underconf_low_conf = []
grand_average_overconf_high_conf = []
grand_average_overconf_low_conf = []

underconf_epochs = strategic_epochs['partner == "underconfident"']
underconf_high_conf_epochs = underconf_epochs['confidence_z_by_participant > 0']
underconf_low_conf_epochs = underconf_epochs['confidence_z_by_participant <= 0']

overconf_epochs = strategic_epochs['partner == "overconfident"']
overconf_high_conf_epochs = overconf_epochs['confidence_z_by_participant > 0']
overconf_low_conf_epochs = overconf_epochs['confidence_z_by_participant <= 0']


for subject in participant_numbers:    
    grand_average_underconf_high_conf.append(underconf_high_conf_epochs['participant == %i' % subject].average())
    grand_average_underconf_low_conf.append(underconf_low_conf_epochs['participant == %i' % subject].average())
    grand_average_overconf_high_conf.append(overconf_high_conf_epochs['participant == %i' % subject].average())
    grand_average_overconf_low_conf.append(overconf_low_conf_epochs['participant == %i' % subject].average())
    

# # plot with 95% confidence intervals
# evokeds = dict(underconf_low_confidence=grand_average_underconf_low_conf,
#                underconf_high_confidence=grand_average_underconf_high_conf,
#                overconf_low_confidence=grand_average_overconf_low_conf,
#                overconf_high_confidence=grand_average_overconf_high_conf)

# mne.viz.plot_compare_evokeds(evokeds, combine='mean', picks=roi, invert_y=False)



# plot without CIs
grand_average_underconf_high_conf = mne.grand_average(grand_average_underconf_high_conf)
grand_average_underconf_low_conf = mne.grand_average(grand_average_underconf_low_conf)
grand_average_overconf_high_conf = mne.grand_average(grand_average_overconf_high_conf)
grand_average_overconf_low_conf = mne.grand_average(grand_average_overconf_low_conf)

evokeds = dict(underconf_low_confidence=grand_average_underconf_low_conf,
               underconf_high_confidence=grand_average_underconf_high_conf,
               overconf_low_confidence=grand_average_overconf_low_conf,
               overconf_high_confidence=grand_average_overconf_high_conf)

print("\n\n strategic task, Pz")
mne.viz.plot_compare_evokeds(evokeds, picks=['Pz'], invert_y=False)
print("\n\n strategic task, roi")
mne.viz.plot_compare_evokeds(evokeds, picks=roi, invert_y=False)
print("\n\n strategic task, most negatively correlating electrodes")
mne.viz.plot_compare_evokeds(evokeds, picks=highest_corrs_neg, invert_y=False)



In [ ]:
# ## STRATEGIC -  z-scoring confidence by partner

# grand_average_underconf_high_conf = []
# grand_average_underconf_low_conf = []
# grand_average_overconf_high_conf = []
# grand_average_overconf_low_conf = []

# underconf_epochs = strategic_epochs['partner == "underconfident"']
# underconf_epochs.metadata['confidence_z_by_participant_by_partner'] = underconf_epochs.metadata['participant_confidence'].groupby(underconf_epochs.metadata['participant']).transform(zscore)
# underconf_high_conf_epochs = underconf_epochs['confidence_z_by_participant_by_partner > 0']
# underconf_low_conf_epochs = underconf_epochs['confidence_z_by_participant_by_partner <= 0']

# overconf_epochs = strategic_epochs['partner == "overconfident"']
# overconf_epochs.metadata['confidence_z_by_participant_by_partner'] = overconf_epochs.metadata['participant_confidence'].groupby(overconf_epochs.metadata['participant']).transform(zscore)
# overconf_epochs = underconf_epochs['confidence_z_by_participant_by_partner > 0']
# overconf_epochs = underconf_epochs['confidence_z_by_participant_by_partner <= 0']


# for subject in participant_numbers:    
#     grand_average_underconf_high_conf.append(underconf_high_conf_epochs['participant == %i' % subject].average())
#     grand_average_underconf_low_conf.append(underconf_low_conf_epochs['participant == %i' % subject].average())
#     grand_average_overconf_high_conf.append(overconf_high_conf_epochs['participant == %i' % subject].average())
#     grand_average_overconf_low_conf.append(overconf_low_conf_epochs['participant == %i' % subject].average())
    

# grand_average_underconf_high_conf = mne.grand_average(grand_average_underconf_high_conf)
# grand_average_underconf_low_conf = mne.grand_average(grand_average_underconf_low_conf)
# grand_average_overconf_high_conf = mne.grand_average(grand_average_overconf_high_conf)
# grand_average_overconf_low_conf = mne.grand_average(grand_average_overconf_low_conf)

# evokeds = dict(underconf_low_confidence=grand_average_underconf_low_conf,
#                underconf_high_confidence=grand_average_underconf_high_conf,
#                overconf_low_confidence=grand_average_overconf_low_conf,
#                overconf_high_confidence=grand_average_overconf_high_conf)

# mne.viz.plot_compare_evokeds(evokeds, picks=['Pz'], invert_y=False)
# mne.viz.plot_compare_evokeds(evokeds, picks=roi, invert_y=False)
# mne.viz.plot_compare_evokeds(evokeds, picks=highest_corrs_neg, invert_y=False)



In [ ]:
# NON-STRATEGIC 

grand_average_underconf_high_conf = []
grand_average_underconf_low_conf = []
grand_average_overconf_high_conf = []
grand_average_overconf_low_conf = []

underconf_epochs = nonstrategic_epochs['partner == "underconfident"']
underconf_high_conf_epochs = underconf_epochs['confidence_z_by_participant > 0']
underconf_low_conf_epochs = underconf_epochs['confidence_z_by_participant <= 0']

overconf_epochs = nonstrategic_epochs['partner == "overconfident"']
overconf_high_conf_epochs = overconf_epochs['confidence_z_by_participant > 0']
overconf_low_conf_epochs = overconf_epochs['confidence_z_by_participant <= 0']


for subject in participant_numbers:    
    grand_average_underconf_high_conf.append(underconf_high_conf_epochs['participant == %i' % subject].average())
    grand_average_underconf_low_conf.append(underconf_low_conf_epochs['participant == %i' % subject].average())
    grand_average_overconf_high_conf.append(overconf_high_conf_epochs['participant == %i' % subject].average())
    grand_average_overconf_low_conf.append(overconf_low_conf_epochs['participant == %i' % subject].average())
    

# # plot with 95% confidence intervals
# evokeds = dict(underconf_low_confidence=grand_average_underconf_low_conf,
#                underconf_high_confidence=grand_average_underconf_high_conf,
#                overconf_low_confidence=grand_average_overconf_low_conf,
#                overconf_high_confidence=grand_average_overconf_high_conf)

# mne.viz.plot_compare_evokeds(evokeds, combine='mean', picks=['Pz'], invert_y=False)



# plot without CIs
grand_average_underconf_high_conf = mne.grand_average(grand_average_underconf_high_conf)
grand_average_underconf_low_conf = mne.grand_average(grand_average_underconf_low_conf)
grand_average_overconf_high_conf = mne.grand_average(grand_average_overconf_high_conf)
grand_average_overconf_low_conf = mne.grand_average(grand_average_overconf_low_conf)

evokeds = dict(underconf_low_confidence=grand_average_underconf_low_conf,
               underconf_high_confidence=grand_average_underconf_high_conf,
               overconf_low_confidence=grand_average_overconf_low_conf,
               overconf_high_confidence=grand_average_overconf_high_conf)

print("\n\n non-strategic task, Pz")
mne.viz.plot_compare_evokeds(evokeds, picks=['Pz'], invert_y=False)
print("\n\n non-strategic task, roi")
mne.viz.plot_compare_evokeds(evokeds, picks=roi, invert_y=False)
print("\n\n non-strategic task, most negatively correlating electrodes")
mne.viz.plot_compare_evokeds(evokeds, picks=highest_corrs_neg, invert_y=False)



In [ ]:
# ## NON-STRATEGIC -  z-scoring confidence by partner

# grand_average_underconf_high_conf = []
# grand_average_underconf_low_conf = []
# grand_average_overconf_high_conf = []
# grand_average_overconf_low_conf = []

# underconf_epochs = nonstrategic_epochs['partner == "underconfident"']
# underconf_epochs.metadata['confidence_z_by_participant_by_partner'] = underconf_epochs.metadata['participant_confidence'].groupby(underconf_epochs.metadata['participant']).transform(zscore)
# underconf_high_conf_epochs = underconf_epochs['confidence_z_by_participant_by_partner > 0']
# underconf_low_conf_epochs = underconf_epochs['confidence_z_by_participant_by_partner <= 0']

# overconf_epochs = nonstrategic_epochs['partner == "overconfident"']
# overconf_epochs.metadata['confidence_z_by_participant_by_partner'] = overconf_epochs.metadata['participant_confidence'].groupby(overconf_epochs.metadata['participant']).transform(zscore)
# overconf_epochs = underconf_epochs['confidence_z_by_participant_by_partner > 0']
# overconf_epochs = underconf_epochs['confidence_z_by_participant_by_partner <= 0']


# for subject in participant_numbers:    
#     grand_average_underconf_high_conf.append(underconf_high_conf_epochs['participant == %i' % subject].average())
#     grand_average_underconf_low_conf.append(underconf_low_conf_epochs['participant == %i' % subject].average())
#     grand_average_overconf_high_conf.append(overconf_high_conf_epochs['participant == %i' % subject].average())
#     grand_average_overconf_low_conf.append(overconf_low_conf_epochs['participant == %i' % subject].average())
    

# grand_average_underconf_high_conf = mne.grand_average(grand_average_underconf_high_conf)
# grand_average_underconf_low_conf = mne.grand_average(grand_average_underconf_low_conf)
# grand_average_overconf_high_conf = mne.grand_average(grand_average_overconf_high_conf)
# grand_average_overconf_low_conf = mne.grand_average(grand_average_overconf_low_conf)

# evokeds = dict(underconf_low_confidence=grand_average_underconf_low_conf,
#                underconf_high_confidence=grand_average_underconf_high_conf,
#                overconf_low_confidence=grand_average_overconf_low_conf,
#                overconf_high_confidence=grand_average_overconf_high_conf)

# mne.viz.plot_compare_evokeds(evokeds, picks=['Pz'], invert_y=False)
# mne.viz.plot_compare_evokeds(evokeds, picks=roi, invert_y=False)
# mne.viz.plot_compare_evokeds(evokeds, picks=highest_corrs_neg, invert_y=False)



### Compute average voltage accross roi for every trial

In [ ]:
roi

In [ ]:
roi_indices = [30, 31, 29, 19, 56]
roi_indices

In [ ]:
#%capture

correlations = []

for subject in participant_numbers:   

    # merge response epochs from both sessions for the participant
    participant_files = []
    for session in sessions:
        participant_files.append('%i_%i' % (subject, session))
    epoch = load_all_eeg(path='%s/' % input_dir, files=participant_files)
    
    # crop epochs
    epoch = epoch.crop(tmin=epoch_start, tmax=epoch_end, include_tmax=True)

    # get epoched data as a numpy array of shape (n_epochs, n_channels, n_times)
    X = epoch.get_data()
    
    Pz = X[:, roi_indices[0], :]
    Pz_trial_average = np.mean(Pz, 1) # average across timepoints
    
    CPz = X[:, roi_indices[1], :]
    CPz_trial_average = np.mean(CPz, 1)
    
    POz = X[:, roi_indices[2], :]
    POz_trial_average = np.mean(POz, 1)
    
    P1 = X[:, roi_indices[3], :]
    P1_trial_average = np.mean(P1, 1)
    
    P2 = X[:, roi_indices[4], :]
    P2_trial_average = np.mean(P2, 1)
    
    Pe_amplitude=[]
    for i in range(Pz_trial_average.shape[0]):
        trial_amplitude = (Pz_trial_average[i]+CPz_trial_average[i]+POz_trial_average[i]+P1_trial_average[i]+P2_trial_average[i])/5
        Pe_amplitude.append(trial_amplitude)
        
    epoch.metadata["Pe_amplitude"] = Pe_amplitude
    
    y = epoch.metadata['participant_confidence'].to_numpy()
    x = epoch.metadata['Pe_amplitude'].to_numpy()
    a = pearsonr(x, y)[0]
    print(a)
    correlations.append(a)
    
    
    df = epoch.metadata
    df.to_csv('metadata_Pe/participant_%i.csv' %subject)

In [ ]:
np.mean(correlations)

### Compare activity between partner conditions at each electrode

In [ ]:
# ALL DATA

underconf_epochs = epochs['partner == "underconfident"']
overconf_epochs = epochs['partner == "overconfident"']

underconf_evokeds = []
overconf_evokeds = []

for subject in participant_numbers:    
    underconf_evokeds.append(underconf_epochs['participant == %i' % subject].average())
    overconf_evokeds.append(overconf_epochs['participant == %i' % subject].average())

# comapre ERPs
evokeds = dict(overconf_partner=overconf_evokeds, underconf_partner=underconf_evokeds)
mne.viz.plot_compare_evokeds(evokeds, combine='mean', picks=['Pz'], invert_y=False, title='Pz, across conditions')

# difference wave
diff_waves = []
for subject in participant_numbers:
    diff_waves.append(mne.combine_evoked([overconf_epochs['participant == %i' % subject].average(), underconf_epochs['participant == %i' % subject].average()],
                                          weights=[1, -1]))
evokeds = dict(difference_wave=diff_waves)
mne.viz.plot_compare_evokeds(evokeds, combine='mean', picks=['Pz'], invert_y=False, title="Pz, difference wave (overconfident-underconfident partner)")


# average topomap for difference wave
mne.viz.plot_evoked_topomap(mne.grand_average(diff_waves), 
                            title="overconfident-underconfident partner, 0.100-0.700s",
                            times=.4, average=0.6,  # .20 to .80
                            size=3)
plt.show()


# t-test
print('\033[1m' + "\n \n - A priori t-test -")
print('\033[0m')
print("testing for difference (overconfident-underconfident partner) to be different from zero at Pz")
# test if ERP effect is significant 
# t-test between pair of conditions on the ERP data averaged over time period of interest and at one or a few electrodes of interest
# ttest_1samp() function from SciPy’s stats module: tests whether a set of values are different from zero
# compute the average over the 200–800 ms time window, for each participant, at electrode Pz, and store these in a NumPy array on which we then perform the t test
evoked_data = np.array([np.mean(e.get_data(picks='Pz', tmin=.100, tmax=.700), axis=1) for e in diff_waves])
t, pval = stats.ttest_1samp(evoked_data, 0)
print('\n Difference t = ', str(round(t[0], 2)), 'p = ', str(round(pval[0], 4)))


# permutation t-test
print('\033[1m' + "\n \n - Permutation t-test -")
print('\033[0m')
print("Perform t-test at every electrode - using permutation t test to control for multiple comparisons")
evoked_data = np.array([np.mean(e.get_data(tmin=.200, tmax=.400), axis=1) for e in diff_waves]) # use all channels
n_permutations = 50000
T0, p_values, H0 = permutation_t_test(evoked_data, n_permutations, tail=-1)
# print("t-values for each channel")
# T0
# print("p-values for each channel (corrected for mulptiple comparions)")
# p_values

print("t-value distribution under the H0 (from the permutations)")
sns.displot(data=H0)
plt.show()

print("top 5% of t-values")
thresh = round(n_permutations * .05)
split = np.argpartition(H0, -thresh)[-thresh:]
t_thresh = sorted(H0[split])[0]

sns.displot(data=H0[split])
plt.ylim(0, 1600)
plt.show()

print("t threshold")
print(t_thresh)

print("\n which electrodes have t-values above threshold")
T0 < -t_thresh

In [ ]:
# STRATEGIC

underconf_epochs = strategic_epochs['partner == "underconfident"']
overconf_epochs = strategic_epochs['partner == "overconfident"']

underconf_evokeds = []
overconf_evokeds = []

for subject in participant_numbers:    
    underconf_evokeds.append(underconf_epochs['participant == %i' % subject].average())
    overconf_evokeds.append(overconf_epochs['participant == %i' % subject].average())

# comapre ERPs
evokeds = dict(overconf_partner=overconf_evokeds, underconf_partner=underconf_evokeds)
mne.viz.plot_compare_evokeds(evokeds, combine='mean', picks=['Pz'], invert_y=False, title='Pz, strategic condition')

# difference wave
diff_waves = []
for subject in participant_numbers:
    diff_waves.append(mne.combine_evoked([overconf_epochs['participant == %i' % subject].average(), underconf_epochs['participant == %i' % subject].average()],
                                          weights=[1, -1]))
evokeds = dict(difference_wave=diff_waves)
mne.viz.plot_compare_evokeds(evokeds, combine='mean', picks=['Pz'], invert_y=False, title="Pz, difference wave (overconfident-underconfident partner)")


# average topomap for difference wave
mne.viz.plot_evoked_topomap(mne.grand_average(diff_waves), 
                            title="overconfident-underconfident partner, 0.100-0.700s",
                            times=.4, average=0.6,  # .20 to .80
                            size=3)
plt.show()


# t-test
print('\033[1m' + "\n \n - A priori t-test -")
print('\033[0m')
print("testing for difference (overconfident-underconfident partner) to be different from zero at Pz")
# test if ERP effect is significant 
# t-test between pair of conditions on the ERP data averaged over time period of interest and at one or a few electrodes of interest
# ttest_1samp() function from SciPy’s stats module: tests whether a set of values are different from zero
# compute the average over the 200–800 ms time window, for each participant, at electrode Pz, and store these in a NumPy array on which we then perform the t test
evoked_data = np.array([np.mean(e.get_data(picks='Pz', tmin=.100, tmax=.700), axis=1) for e in diff_waves])
t, pval = stats.ttest_1samp(evoked_data, 0)
print('\n Difference t = ', str(round(t[0], 2)), 'p = ', str(round(pval[0], 4)))


# permutation t-test
print('\033[1m' + "\n \n - Permutation t-test -")
print('\033[0m')
print("Perform t-test at every electrode - using permutation t test to control for multiple comparisons")
evoked_data = np.array([np.mean(e.get_data(tmin=.200, tmax=.400), axis=1) for e in diff_waves]) # use all channels
n_permutations = 50000
T0, p_values, H0 = permutation_t_test(evoked_data, n_permutations, tail=-1)
# print("t-values for each channel")
# T0
# print("p-values for each channel (corrected for mulptiple comparions)")
# p_values

print("t-value distribution under the H0 (from the permutations)")
sns.displot(data=H0)
plt.show()

print("top 5% of t-values")
thresh = round(n_permutations * .05)
split = np.argpartition(H0, -thresh)[-thresh:]
t_thresh = sorted(H0[split])[0]

sns.displot(data=H0[split])
plt.ylim(0, 1600)
plt.show()

print("t threshold")
print(t_thresh)

print("\n which electrodes have t-values above threshold")
T0 < -t_thresh

In [ ]:
# NON-STRATEGIC

underconf_epochs = nonstrategic_epochs['partner == "underconfident"']
overconf_epochs = nonstrategic_epochs['partner == "overconfident"']

underconf_evokeds = []
overconf_evokeds = []

for subject in participant_numbers:    
    underconf_evokeds.append(underconf_epochs['participant == %i' % subject].average())
    overconf_evokeds.append(overconf_epochs['participant == %i' % subject].average())

# comapre ERPs
evokeds = dict(overconf_partner=overconf_evokeds, underconf_partner=underconf_evokeds)
mne.viz.plot_compare_evokeds(evokeds, combine='mean', picks=['Pz'], invert_y=False, title='Pz, non-strategic condition')

# difference wave
diff_waves = []
for subject in participant_numbers:
    diff_waves.append(mne.combine_evoked([overconf_epochs['participant == %i' % subject].average(), underconf_epochs['participant == %i' % subject].average()],
                                          weights=[1, -1]))
evokeds = dict(difference_wave=diff_waves)
mne.viz.plot_compare_evokeds(evokeds, combine='mean', picks=['Pz'], invert_y=False, title="Pz, difference wave (overconfident-underconfident partner)")


# average topomap for difference wave
mne.viz.plot_evoked_topomap(mne.grand_average(diff_waves), 
                            title="overconfident-underconfident partner, 0.100-0.700s",
                            times=.4, average=0.6,  # .20 to .80
                            size=3)
plt.show()


# t-test
print('\033[1m' + "\n \n - A priori t-test -")
print('\033[0m')
print("testing for difference (overconfident-underconfident partner) to be different from zero at Pz")
# test if ERP effect is significant 
# t-test between pair of conditions on the ERP data averaged over time period of interest and at one or a few electrodes of interest
# ttest_1samp() function from SciPy’s stats module: tests whether a set of values are different from zero
# compute the average over the 200–800 ms time window, for each participant, at electrode Pz, and store these in a NumPy array on which we then perform the t test
evoked_data = np.array([np.mean(e.get_data(picks='Pz', tmin=.100, tmax=.700), axis=1) for e in diff_waves])
t, pval = stats.ttest_1samp(evoked_data, 0)
print('\n Difference t = ', str(round(t[0], 2)), 'p = ', str(round(pval[0], 4)))


# permutation t-test
print('\033[1m' + "\n \n - Permutation t-test -")
print('\033[0m')
print("Perform t-test at every electrode - using permutation t test to control for multiple comparisons")
evoked_data = np.array([np.mean(e.get_data(tmin=.200, tmax=.400), axis=1) for e in diff_waves]) # use all channels
n_permutations = 50000
T0, p_values, H0 = permutation_t_test(evoked_data, n_permutations, tail=-1)
# print("t-values for each channel")
# T0
# print("p-values for each channel (corrected for mulptiple comparions)")
# p_values

print("t-value distribution under the H0 (from the permutations)")
sns.displot(data=H0)
plt.show()

print("top 5% of t-values")
thresh = round(n_permutations * .05)
split = np.argpartition(H0, -thresh)[-thresh:]
t_thresh = sorted(H0[split])[0]

sns.displot(data=H0[split])
plt.ylim(0, 1600)
plt.show()

print("t threshold")
print(t_thresh)

print("\n which electrodes have t-values above threshold")
T0 < -t_thresh

### Compare activity for correct/incorrect responses

In [ ]:
# ALL DATA

correct_epochs = epochs['participant_correct == True']
incorrect_epochs = epochs['participant_correct == False']

correct_evokeds = []
incorrect_evokeds = []

for subject in participant_numbers:    
    correct_evokeds.append(correct_epochs['participant == %i' % subject].average())
    incorrect_evokeds.append(incorrect_epochs['participant == %i' % subject].average())

# comapre ERPs
evokeds = dict(correct=correct_evokeds, incorrect=incorrect_evokeds)
mne.viz.plot_compare_evokeds(evokeds, combine='mean', picks=['Pz'], invert_y=False, title='Pz, across conditions')

# difference wave
diff_waves = []
for subject in participant_numbers:
    diff_waves.append(mne.combine_evoked([correct_epochs['participant == %i' % subject].average(), incorrect_epochs['participant == %i' % subject].average()],
                                          weights=[-1, 1]))
evokeds = dict(difference_wave=diff_waves)
mne.viz.plot_compare_evokeds(evokeds, combine='mean', picks=['Pz'], invert_y=False, title="Pz, difference wave (incorrect-correct)")


# average topomap for difference wave
mne.viz.plot_evoked_topomap(mne.grand_average(diff_waves), 
                            title="correct-incorrect, 0.100-0.700s",
                            times=.4, average=0.6,  # .20 to .80
                            size=3)
plt.show()


# t-test
print('\033[1m' + "\n \n - A priori t-test -")
print('\033[0m')
print("testing for difference (correct-incorrect) to be different from zero at Pz")
# test if ERP effect is significant 
# t-test between pair of conditions on the ERP data averaged over time period of interest and at one or a few electrodes of interest
# ttest_1samp() function from SciPy’s stats module: tests whether a set of values are different from zero
# compute the average over the 200–800 ms time window, for each participant, at electrode Pz, and store these in a NumPy array on which we then perform the t test
evoked_data = np.array([np.mean(e.get_data(picks='Pz', tmin=.100, tmax=.700), axis=1) for e in diff_waves])
t, pval = stats.ttest_1samp(evoked_data, 0)
print('\n Difference t = ', str(round(t[0], 2)), 'p = ', str(round(pval[0], 4)))


# permutation t-test
print('\033[1m' + "\n \n - Permutation t-test -")
print('\033[0m')
print("Perform t-test at every electrode - using permutation t test to control for multiple comparisons")
evoked_data = np.array([np.mean(e.get_data(tmin=.200, tmax=.400), axis=1) for e in diff_waves]) # use all channels
n_permutations = 50000
T0, p_values, H0 = permutation_t_test(evoked_data, n_permutations, tail=-1)
# print("t-values for each channel")
# T0
# print("p-values for each channel (corrected for mulptiple comparions)")
# p_values

print("t-value distribution under the H0 (from the permutations)")
sns.displot(data=H0)
plt.show()

print("top 5% of t-values")
thresh = round(n_permutations * .05)
split = np.argpartition(H0, -thresh)[-thresh:]
t_thresh = sorted(H0[split])[0]

sns.displot(data=H0[split])
plt.ylim(0, 1600)
plt.show()

print("t threshold")
print(t_thresh)

print("\n which electrodes have t-values above threshold")
T0 < -t_thresh

### Understanding how to get the sensor projections

In [ ]:
x = np.array([[1, 4],[2, 5], [3, 6]])
y = np.array([1, 2, 3])

In [ ]:
a, err, _, __ = np.linalg.lstsq(x, y, rcond=None)
a

In [ ]:
# should give [1, 2.2857] (jasmine just uses a = y_pred \ X_test)
a = y.T@x * ( 1/ (y.T@y) )
a